# lib

In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] ="expandable_segments:True"
import gymnasium as gym

#import gym
import numpy as np

import collections
import pickle
import tqdm

from stable_baselines3.common.buffers import ReplayBuffer


import os
import random
import time
from dataclasses import dataclass

import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import tyro
from torch.distributions.categorical import Categorical
from torch.utils.tensorboard import SummaryWriter

from typing import Optional

import functools
import random
from copy import copy

import numpy as np
from gymnasium.spaces import Discrete, MultiDiscrete, Box, Dict

from pettingzoo import AECEnv



import gymnasium


from pettingzoo.utils import agent_selector, wrappers

from gymnasium.utils import EzPickle



from statistics import NormalDist

import pygame

from typing import Any , Generic, Iterable, Iterator, TypeVar
ActionType = TypeVar("ActionType")

import collections
from torch.utils.data import Dataset, DataLoader


C:\Users\31721\.conda\envs\DL_project\lib\site-packages\pkg_resources\__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
C:\Users\31721\.conda\envs\DL_project\lib\site-packages\pkg_resources\__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
C:\Users\31721\.conda\envs\DL_project\lib\site-packages\pkg_resources\__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-

In [2]:
from Utilities.new_models import *
from Utilities.Transformer_risk_act_2 import *
import utils_gym
import env_model_class_2


from board_env import *



%load_ext autoreload
%autoreload 2

In [3]:
from IPython.display import clear_output, display

In [4]:
#torch.autograd.set_detect_anomaly(False)

#  Algorithm module definition 

In [5]:
class DDQN_module:
    def __init__(self, qnet_config_dict, actor_config_dict,args,device,writer,run_name,agent):

        self.agent = agent
        self.run_name =run_name 
        self.actor_config_dict = actor_config_dict
        self.qnet_config_dict = qnet_config_dict
        self.args = args
        self.device = device
        self.writer = writer
        self.entropy=args.entropy
        self.return_prob=args.return_prob
        self.actor_wt = args.actor_wt
        self.CE_wt = args.CE_wt
        if self.args.small:
            self.actor = Actor_ddqn_small(**self.actor_config_dict).to(self.device)
            self.qf1 = QNetwork_small(**self.qnet_config_dict).to(self.device)
            self.qf1_target = QNetwork_small(**self.qnet_config_dict).to(self.device)
            self.target_actor = Actor_ddqn_small(**self.actor_config_dict).to(self.device)
            
        else:
            self.actor = Actor_ddqn(**self.actor_config_dict).to(self.device)
            self.qf1 = QNetwork(**self.qnet_config_dict).to(self.device)
            self.qf1_target = QNetwork(**self.qnet_config_dict).to(self.device)
            self.target_actor = Actor_ddqn(**self.actor_config_dict).to(self.device)
            
            
        
        
        self.target_actor.load_state_dict(self.actor.state_dict())
        self.qf1_target.load_state_dict(self.qf1.state_dict())
        self.q_optimizer = optim.Adam(list(self.qf1.parameters()), lr=self.args.learning_rate)
        self.actor_optimizer = optim.Adam(list(self.actor.parameters()), lr=self.args.learning_rate)

    def action_predict(self,data):
        return self.actor(data)

    def train_write(self,data,iteration,epoch):
        #data = rb.sample(self.args.batch_size)
        qf1_a_values, qf1_loss, actor_loss = self.train(data)
        self.write(qf1_a_values, qf1_loss, actor_loss,epoch,iteration)

    def cal_q_loss(self,qf1_a_values, next_q_value):
        criterion = nn.SmoothL1Loss()
        
        return criterion(qf1_a_values, next_q_value)
        #return nn.functional.mse_loss(qf1_a_values, next_q_value)

    
    def cal_actor_loss(self,data,entropy=False,return_prob=False):

        if not entropy:
            return -self.qf1(data.observations[:,:-1], self.actor(data.observations[:,:-1])).mean()
        else:
            actions,probs = self.actor(data.observations[:,:-1],return_prob=return_prob)
            action_loss = -self.qf1(data.observations[:,:-1],actions).mean()

            entropy_ = -(probs*torch.log2(probs)).sum(-1).mean()
            return self.actor_wt*action_loss + self.CE_wt*entropy_# if not torch.isnan(cross_entropy_loss) else 0)
        

    def train(self,data):


        
        with torch.no_grad():
            collected_t_next = data.next_observations[:,-1]
            next_state_actions = self.target_actor(data.next_observations[:,:-1])
            qf1_next_target = self.qf1_target(data.next_observations[:,:-1], next_state_actions)

            
            next_q_value = data.rewards.flatten() + (1 - data.dones.flatten()) * (self.args.gamma**(collected_t_next+1)).view(-1) * (qf1_next_target).view(-1)
    
        qf1_a_values = self.qf1(data.observations[:,:-1], data.actions).view(-1)
        qf1_loss = self.cal_q_loss(qf1_a_values, next_q_value)
        
        # optimize the model
        self.q_optimizer.zero_grad()
        qf1_loss.backward()
        self.q_optimizer.step()
        
        #if global_step % args.policy_frequency == 0:
        actor_loss = self.cal_actor_loss(data,entropy=self.entropy,return_prob=self.return_prob)
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()
    
        # update the target network
        for param, target_param in zip(self.actor.parameters(), self.target_actor.parameters()):
            target_param.data.copy_(self.args.tau * param.data + (1 - self.args.tau) * target_param.data)
        for param, target_param in zip(self.qf1.parameters(), self.qf1_target.parameters()):
            target_param.data.copy_(self.args.tau * param.data + (1 - self.args.tau) * target_param.data)
        return qf1_a_values, qf1_loss, actor_loss
        

    def write(self,qf1_a_values, qf1_loss, actor_loss,epoch,iteration):
        
        ind_epoch = epoch + (iteration-1)*self.args.update_epochs
        self.writer.add_scalar(f"losses/{self.agent}/qf1_values", qf1_a_values.mean().item(), ind_epoch)
        
        self.writer.add_scalar(f"losses/{self.agent}/qf1_loss", qf1_loss.item(), ind_epoch)
        self.writer.add_scalar(f"losses/{self.agent}/actor_loss", actor_loss.item(), ind_epoch)
        
    def save_models(self):
        newpath = r'./models/'+ self.run_name +'/'+str(self.agent)
        if not os.path.exists(newpath):
            os.makedirs(newpath)
        torch.save(self.actor.state_dict(), newpath+"/actor.pt")
        torch.save(self.qf1.state_dict(), newpath+"/qf1.pt")
        torch.save(self.qf1_target.state_dict(), newpath+"/qf1_target.pt")
        torch.save(self.target_actor.state_dict(), newpath+"/target_actor.pt")       
    

    def load_models(self):
        newpath = r'./models/'+ self.run_name +'/'+str(self.agent)
        self.actor.load_state_dict(torch.load(newpath+"/actor.pt"))
        self.qf1.load_state_dict(torch.load(newpath+"/qf1.pt"))
        self.qf1_target.load_state_dict(torch.load(newpath+"/qf1_target.pt"))
        self.target_actor.load_state_dict(torch.load(newpath+"/target_actor.pt")) 
        

                                          
                   

## transfomer

In [6]:
#		s1,a1,r1,s2 = self.ram.sample(BATCH_SIZE)
#
#		s1 = Variable(torch.from_numpy(s1))
#		a1 = Variable(torch.from_numpy(a1))
#		r1 = Variable(torch.from_numpy(r1))
#		s2 = Variable(torch.from_numpy(s2))
#
#		# ---------------------- optimize critic ----------------------
#		# Use target actor exploitation policy here for loss evaluation
#		a2 = self.target_actor.forward(s2).detach()
#		next_val = torch.squeeze(self.target_critic.forward(s2, a2).detach())
#		# y_exp = r + gamma*Q'( s2, pi'(s2))
#		y_expected = r1 + GAMMA*next_val
#		# y_pred = Q( s1, a1)
#		y_predicted = torch.squeeze(self.critic.forward(s1, a1))
#		# compute critic loss, and update the critic
#		loss_critic = F.smooth_l1_loss(y_predicted, y_expected)
#		self.critic_optimizer.zero_grad()
#		loss_critic.backward()
#		self.critic_optimizer.step()
#
#		# ---------------------- optimize actor ----------------------
#		pred_a1 = self.actor.forward(s1)
#		loss_actor = -1*torch.sum(self.critic.forward(s1, pred_a1))
#		self.actor_optimizer.zero_grad()
#		loss_actor.backward()
#		self.actor_optimizer.step()
#
#		utils.soft_update(self.target_actor, self.actor, TAU)
#		utils.soft_update(self.target_critic, self.critic, TAU)
#z
#		# if self.iter % 100 == 0:
#		# 	print 'Iteration :- ', self.iter, ' Loss_actor :- ', loss_actor.data.numpy(),#		# 		' Loss_critic :- ', loss_critic.data.numpy()
#		# self.iter += 1

In [7]:
#!/usr/bin/python
# -*- coding: utf-8 -*-


#referrence for loss : https://github.com/vy007vikas/PyTorch-ActorCriticRL/blob/master/train.py

#reference invalid actions ignore
#https://ai.stackexchange.com/questions/2980/how-should-i-handle-invalid-actions-when-using-reinforce
#ttps://arxiv.org/abs/2006.14171


class transformer_model:

    def __init__(  # ,config,args,hero = 1,
        self,
        qnet_config_dict,
        actor_config_dict,
        args,
        device,
        writer,
        run_name,
        agent,
        ):

        self.writer = writer
        self.hero = agent
        self.args = args
        self.run_name = run_name
        self.device = device  # config['device']
        self.state_dim = actor_config_dict['ob_space']  # config['observation_space']#.shape[0]
        self.act_dim = actor_config_dict['action_space']  # config['action_space']#.n #3#1 #env.action_space.shape[0]
        self.n_blocks = args.model_config['n_blocks']
        self.embed_dim = args.model_config['embed_dim']
        self.context_len = args.model_config['context_len']
        self.n_heads = args.model_config['n_heads']
        self.dropout_p = args.model_config['dropout_p']
        self.lr = args.learning_rate
        self.wt_decay = args.model_config['wt_decay']
        self.rb_len = args.model_config['rb_len']

        # self.steps        =            config['steps']

        self.warmup_epoch = args.model_config['warmup_epoch']
        self.total_epoch = args.model_config['total_epoch']
        self.initial_lr =  args.model_config['initial_lr'] #5e-4
        self.final_lr =  args.model_config['final_lr'] #1e-6
        
        self.chunk_size = args.model_config['chunk_size']
        self.chunk_overlap = args.model_config['chunk_overlap']
        #self.max_d4rl_score = -1000.0
        #self.total_updates = 0
        self.tau = args.model_config['tau']
        self.num_steps = args.num_steps
        self.total_agents = args.total_agents
        self.total_phases = args.total_phases
        self.beta = args.model_config['beta']         #0.2 #Q_mse
        self.alpha =args.model_config['alpha']          #0.1  #actionloss
        self.entropy_coeff = args.model_config['entropy_coeff']         #0.1#0.5   #entropy loss in action
        self.val_loss_coeff = args.model_config['val_loss_coeff']        #0.5      #Q loss
        

        # context_len_=200

        self.model = DecisionTransformer(
            state_dim=self.state_dim,
            act_dim=self.act_dim,
            n_blocks=self.n_blocks,
            h_dim=self.embed_dim,
            context_len=self.context_len,
            n_heads=self.n_heads,
            drop_p=self.dropout_p,
            max_timestep=self.num_steps,
            ).to(self.device)

        self.target_model = DecisionTransformer(
            state_dim=self.state_dim,
            act_dim=self.act_dim,
            n_blocks=self.n_blocks,
            h_dim=self.embed_dim,
            context_len=self.context_len,
            n_heads=self.n_heads,
            drop_p=self.dropout_p,
            max_timestep=self.num_steps,
            ).to(self.device)

        # Set target network parameters to not require gradients
        for param in self.target_model.parameters():
            param.requires_grad = False

        self.copy_wt()
        self.optimizer_2 = torch.optim.AdamW(self.model.parameters(),
                lr=0.000005, weight_decay=self.wt_decay)  # lr,

        # lr = 0.00001
        
        self.optimizer_1 = torch.optim.AdamW(self.model.parameters(),
                lr=self.lr, weight_decay=self.wt_decay)  # lr,

        #self.scheduler =             torch.optim.lr_scheduler.LambdaLR(self.optimizer_1, lambda steps: min((steps + 1) / self.warmup_steps, 1))
        self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(self.optimizer_1,T_max = self.total_epoch - self.warmup_epoch, eta_min=self.final_lr)
        # scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer_1, T_0=150, T_mult=2, eta_min=0.01, last_epoch=-1)


    













    

    def copy_wt(self):

        # target_param.load_state_dict(param.state_dict())

        for (param, target_param) in zip(self.model.parameters(),
                self.target_model.parameters()):
            target_param.data.copy_(self.tau * param.data + (1
                                    - self.tau) * target_param.data)


    def action_loss_fn_3(
        self,
        timesteps,
        states,
        actions_1,actions_2,log_probs_actions_2,
        returns_to_go,
        return_preds_last,
        returns_to_go_cal_last,
        print_,
        action_masks
        ):

                         # action_preds_2,
                         # action_mask,
                         # return_preds_2,
                         # returns_target,
                         # beta = 0.2

        # model : calculate the action for the states
        # create a new dataset with last action replaced by a_pred_module
        # predict value of the action using model critic
        # torch sum value....

        # pred_a1 = self.actor.forward(s1)
        # loss_actor = -1*torch.sum(self.critic.forward(s1, pred_a1))
        # self.actor_optimizer.zero_grad()
        # loss_actor.backward()
        # self.actor_optimizer.step()

        # so what should i do ..... hmmm ... yeah i only need time steps where agent is the hero agent...

        ################################
        #no epoch no re prediction .... 
        
    
        if True: # this does not look into old values
            ( #_, 
             action_logit_model_1,#action_model_2_dir
                _,_,logp_pi_a_2_dir,dist_entropy_a_1) =   self.model.forward(timesteps=timesteps, states=states,
                                   actions_1=actions_1,actions_2=actions_2,
                                   returns_to_go=returns_to_go, print_=2,return_logit=True,return_og_log_prob_a2=True)  # print_
    
                                                            # ,info = info
    
            actions_1_ = actions_1.clone().detach()#, requires_grad=False)
    
            # self.actions_ = actions_
            # self.action_preds_model_ = action_preds_model
            # actions_[:,:,-1] = action_preds_model
    

            log_probs_1 = -torch.nn.functional.cross_entropy(action_logit_model_1[:, -1, :], 
                                           actions_1_[:, -1].long(), reduction="none")
            
            #(a1_[:,-1,:][hero_steps[:,-1,0]]*action_masks[:,-1,:][hero_steps[:,-1,0]])



            
            #actions_1_[:, -1] =     ((action_logit_model_1[:, -1, :] + 0.000001*action_masks) *action_masks).argmax(axis=1).clone().detach() #i will fix this later, there is an issue.... when probabilities for all legal actions are 0, the product is same as illigal actions 
            actions_1_[:, -1] =     (action_logit_model_1[:, -1, :] *action_masks).argmax(axis=1).clone().detach() #i will fix this later, there is an issue.... when probabilities for all legal actions are 0, the product is same as illigal actions 

            ( #_, 
             _,#action_logit_model_2
                _,_,logp_pi_a_2,_,) =   self.model.forward(timesteps=timesteps, states=states,
                                   actions_1=actions_1_,actions_2=actions_2,
                                   returns_to_go=returns_to_go, print_=2,return_logit=True,return_og_log_prob_a2=True)
            

            
            #if False:
            #with torch.no_grad():
            #    ( #_, 
            #     _,_,values_) =   self.model.forward(timesteps=timesteps, states=states,
            #                       actions_1=actions_1_,actions_2=actions_2,
            #                       returns_to_go=returns_to_go, print_=2,return_logit=False)

            advantages  = returns_to_go_cal_last - return_preds_last.clone().detach()
            
            



            
            #if (not torch.isfinite(action_logit_model_1[:, -1,:]).all()) :
            #    print('inf_error', 'action_logit_model_1')
            #    print('action_logit_model_1',action_logit_model_1[:, -1,:])
            #    print('log_probs_1',log_probs_1)
            #    print('actions_1_',actions_1_[:, -1].long())
            #    a()
            #    
            #if (not torch.isfinite(log_probs_1).all()):
            #    print('inf_error', 'log_probs_1')
            #    print('action_logit_model_1',action_logit_model_1[:, -1,:])
            #    print('log_probs_1',log_probs_1)
            #    print('actions_1_',actions_1_[:, -1].long())
            #    a()
            #    
            #if (not torch.isfinite(actions_1_[:, -1].long() ).all())  :
            #    print('inf_error','actions_1_')
            #    print('action_logit_model_1',action_logit_model_1[:, -1,:])
            #    print('log_probs_1',log_probs_1)
            #    print('actions_1_',actions_1_[:, -1].long())
            #    a()
                
                
                
            
            try:
                #the model is sending everyone .... 1. .... there must be an issue .... lower the weight
                log_probs_2_dir = -torch.nn.BCELoss(reduction ='none')(

                                               #logp_pi_a_2_dir[:, -1,0],
                                               torch.clamp(logp_pi_a_2_dir[:, -1,0], min=0.00001, max=0.9999),
                                               #torch.clamp(action_model_2_dir[:, -1,0], min=0.00001, max=0.9999), 
                                               torch.clamp(log_probs_actions_2[:, -1], min=0.00001, max=0.9999))

                #if (not torch.isfinite(action_model_2_dir[:, -1,0]).all()) or (action_model_2_dir[:, -1,0] ==1).any() or (action_model_2_dir[:, -1,0] ==0).any():
                #    print('inf_error', 'action_model_2_dir')
                #    print('action_model_2_dir',action_model_2_dir[:, -1,0])
                #    print('log_probs_2_dir',log_probs_2_dir)
                #    print('actions_2',actions_2[:, -1])
                #    a()
                #    
                #if (not torch.isfinite(log_probs_2_dir).all()):
                #    print('inf_error', 'log_probs_1')
                #    print('action_model_2_dir',action_model_2_dir[:, -1,0])
                #    print('log_probs_2_dir',log_probs_2_dir)
                #    print('actions_2',actions_2[:, -1])
                #    a()
                #
                #if (not torch.isfinite(actions_2[:, -1]).all()):
                #    print('inf_error', 'actions_2')
                #    print('action_model_2_dir',action_model_2_dir[:, -1,0])
                #    print('log_probs_2_dir',log_probs_2_dir)
                #    print('actions_2',actions_2[:, -1])
                #    a()




            
            except Exception as e:
                print(e)
                print(logp_pi_a_2_dir[:, -1,0])
                print(log_probs_actions_2[:, -1])
                a()

            log_probs_2 = -torch.nn.BCELoss(reduction ='none')(#logp_pi_a_2[:, -1,0],#action_logit_model_2[:, -1,0], 
                                                               torch.clamp(logp_pi_a_2[:, -1,0], min=0.00001, max=0.9999),
                                           torch.clamp(log_probs_actions_2[:, -1], min=0.00001, max=0.9999)
                                            )
            
            #if (not torch.isfinite(log_probs_2).all()):
            #    #pass
            #    print('inf_error', 'log_probs_2')
            #    print('log_probs_2',log_probs_2)
            #    a()

            #if (not torch.isfinite(action_logit_model_2[:, -1,0]).all()) or (action_logit_model_2[:, -1,0] ==1).any() or (action_logit_model_2[:, -1,0] ==0).any():
            #        print('inf_error', 'action_logit_model_2')
            #        print('action_logit_model_2',action_logit_model_2[:, -1,0])
            #        a()

            #print(logp_pi_a_2_dir[:,-1,0],logp_pi_a_2[:,-1,0])

            pi_loss = -((log_probs_1 +log_probs_2 + log_probs_2_dir)*(advantages)).mean() - self.entropy_coeff*dist_entropy_a_1.mean()

            
            
            return pi_loss

    def value_loss_fn_3(
        self,
        reward_last,
        return_preds_last,
        returns_target_last,
        returns_to_go_cal_last,
        beta=0.2,
        alpha=2,
        gamma=0.99,
        device='cpu',
        ):
        
        RT1 = reward_last[:-1] + gamma * returns_target_last[1:]
        try:
            Q_TD = torch.nn.functional.smooth_l1_loss(return_preds_last[:-1],RT1) 
            
            Q_TD=Q_TD+torch.nn.functional.smooth_l1_loss(return_preds_last[[-1]], reward_last[[-1]])
            
            Q_MSE = torch.nn.functional.smooth_l1_loss(return_preds_last, returns_to_go_cal_last)
            return (Q_TD ,  Q_MSE)
        except Exception as e:
            print(e)
            print('in-v-loss')
            print( 'return_preds_last' ,return_preds_last)
            print('RT1' ,RT1)
            print('reward_last' ,reward_last)
            print('returns_to_go_cal_last' ,returns_to_go_cal_last)
            a()

        

    def calculate_loss(
        self,
        timesteps,
        returns_target, #target model predictions 
        returns_to_go_cal,  # Q calculated
        hero_steps,#current_agent_acting,
        states,
        actions_1,actions_2,log_probs_actions_2,
        return_preds,
        return_preds_v        
        ,reward, #actual rewards 
        returns_to_go,  #actual R2G given to the model
        action_masks,
        print_,
        chunk_id,
        divi
        ):


        
        # only consider non padded elements

        reward_last = reward[:, -1].squeeze().view(-1).to(self.device,dtype=torch.float32) #require the last reward only
        return_preds_last = return_preds[:, -1,-1].squeeze().view(-1).to(self.device,dtype=torch.float32)  # act_dim , this is what out model predicted
        return_preds_v_last = return_preds_v[:, -1,-1].squeeze().view(-1).to(self.device,dtype=torch.float32)  # act_dim , this is what out model predicted
       
        returns_target_last = returns_target[:, -1,-1].squeeze().view(-1).to(self.device,dtype=torch.float32) # target model prediction ... should be the 1st prediction
       
        returns_to_go_cal_last = returns_to_go_cal[:, -1,-1].squeeze().view(-1).to(self.device,dtype=torch.float32) # we need the Q value of the last action


        
        
        hero_step_filter = hero_steps[:,-1,0] # current_agent_acting[:, -1, 0] == self.hero #last action is out heros

        if sum(hero_step_filter) == 0:
            policy_loss = torch.tensor(0,device=self.device)
        else:
            policy_loss = self.action_loss_fn_3(timesteps[hero_step_filter],
                                                states[hero_step_filter],
                                                actions_1[hero_step_filter],actions_2[hero_step_filter],log_probs_actions_2[hero_step_filter],
                                                returns_to_go[hero_step_filter],
                                                
                                                return_preds_last[hero_step_filter],
                                                
                                                returns_to_go_cal_last[hero_step_filter], print_,
                                               
                                                action_masks[:,-1,:][hero_step_filter])

        # but we need to figure out a little more



        if  len(reward_last) == 0:
            print('chunk_id',chunk_id)
            print('reward',reward.shape)
            print('reward_last',reward)
            print('return_preds_last',return_preds_last.shape)


            print('return_preds_last',return_preds_last)
            
            





        
        if True:  # self.value_cal_loss:
            (Q_TD , Q_MSE) = self.value_loss_fn_3(  # value_loss_fn(return_preds_2,return_target,action_mask,beta=1)
                                    reward_last,
                                    return_preds_v_last,
                                    returns_target_last,
                                    returns_to_go_cal_last,
                                    beta=0.5,
                                    alpha=2,
                                    gamma=0.99
                                )

            Q_TD = Q_TD/divi
            Q_MSE = Q_MSE/divi
            Q_loss = Q_TD + self.beta*Q_MSE
            
            policy_loss =  policy_loss/divi

            
            total_loss = self.val_loss_coeff*Q_loss + self.alpha*policy_loss

            if (not torch.isfinite(policy_loss)) or (not torch.isfinite(Q_TD)) or (not torch.isfinite(Q_MSE)):
                print(divi, 'policy_loss',policy_loss,'Q_TD',Q_TD,'Q_MSE',Q_MSE)
                a()
            
            


        return total_loss, (Q_TD, Q_MSE, Q_loss, policy_loss)

    def train_write(self, iteration, print_=False):

        total_loss_list = []
        Q_TD_list = []
        Q_MSE_list = []
        Q_loss_list = []
        policy_loss_list = []

        if self.args.TB_log:
            self.writer.add_scalar("charts/learning_rate", self.optimizer_1.param_groups[0]["lr"], iteration)        
        
        for epoch in range(self.args.update_epochs):
            for i,batch in enumerate(self.traj_data_loader):
                total_loss = 0
                Q_TD = 0
                Q_MSE = 0
                Q_loss = 0
                policy_loss = 0
                print(i)


                batch_len = batch[0].shape[1]#2840
                
                if batch_len > self.chunk_size:
                    a_ = [(i,i+self.chunk_size) for i in range(0, batch_len     -self.chunk_size,self.chunk_size-self.chunk_overlap)  ]
                    a_  =a_+[ (a_[-1][1] -self.chunk_overlap ,batch_len) ]
                else:
                    a_ = [(0,batch_len)]
    


                

                divi = len(a_)

                
                print('divi',divi,'chunk_size',self.chunk_size,'batch_shape',batch[0].shape)
                
                for (chunk_id, i) in enumerate(a_):#range(0,batch[0].shape[1] - self.chunk_size + 1,self.chunk_size - self.chunk_overlap)):

                    # print(i,(i + self.chunk_size))
                    total_loss_chunk, Q_TD_chunk, Q_MSE_chunk, Q_loss_chunk, policy_loss_chunk = self.train_write_smaller_chunk((tens[:, i[0]:i[1]] for tens in batch),
                                                                                                                                    iteration, epoch, chunk_id, print_=print_,divi=divi)

                    
                    total_loss= total_loss + total_loss_chunk
                    Q_TD = Q_TD+ Q_TD_chunk
                    Q_MSE = Q_MSE+ Q_MSE_chunk
                    Q_loss = Q_loss+ Q_loss_chunk
                    policy_loss = policy_loss+ policy_loss_chunk

                total_loss_list.append(total_loss)
                Q_TD_list.append(Q_TD)
                Q_MSE_list.append(Q_MSE)
                Q_loss_list.append(Q_loss)
                policy_loss_list.append(policy_loss)            
                
                torch.cuda.empty_cache()

        
        
        if iteration < self.warmup_epoch:
            # Linear warmup: Gradually increase learning rate during warmup
            lr = self.initial_lr * iteration / self.warmup_epoch
            for param_group in self.optimizer_1.param_groups:
                param_group['lr'] = lr
        else:
            self.scheduler.step()

        
        print("total_loss", np.mean(total_loss_list),
              "Q_TD", np.mean(Q_TD_list),
                "Q_MSE", np.mean(Q_MSE_list),
                "Q_loss", np.mean(Q_loss_list),
                "policy_loss", np.mean(policy_loss_list))

        if self.args.TB_log:
        
            self.writer.add_scalar("total_loss", np.mean(total_loss_list), iteration)
            self.writer.add_scalar("Q_TD", np.mean(Q_TD_list), iteration)
            self.writer.add_scalar("Q_MSE", np.mean(Q_MSE_list), iteration)
            self.writer.add_scalar("Q_loss", np.mean(Q_loss_list), iteration)
            self.writer.add_scalar("policy_loss", np.mean(policy_loss_list), iteration)    
                    
                
                
        
        self.copy_wt()

    def train_write_smaller_chunk(
        self,
        data,
        iteration,
        epoch,
        chunk_id,
        print_=False,divi = 1
        ):

        (
            timesteps,
            states,
            actions_1,actions_2,log_probs_actions_2,
            returntogo,
            returns_to_go_cal,
            returntogo_pred,
            reward,
            traj_mask,
            action_masks,
            current_agent_acting,
            current_agent_simple,
            current_agent,
            current_phase,
            current_troops_count,
            ) = data


        if len(timesteps[0].shape) ==0:
            print( timesteps.shape )

        
        timesteps = timesteps[0].to(self.device)  # B x T
        states = states[0].to(self.device)  # B x T x state_dim
        actions_1 = actions_1[0].to(self.device)  # B x T x act_dim
        actions_2 = actions_2[0].to(self.device)  # B x T x act_dim
        log_probs_actions_2 = log_probs_actions_2[0].to(self.device)
        reward = reward[0].to(self.device)
        returns_to_go_cal =             returns_to_go_cal[0].to(self.device).unsqueeze(dim=-1)  # B x T x 1
        returntogo = returntogo[0].to(self.device)
        returntogo_pred = returntogo_pred[0].to(self.device)
        traj_mask = traj_mask[0].to(self.device)  # B x T
        action_masks = action_masks[0].to(self.device)
        current_agent_acting = current_agent_acting[0].to(self.device)
        current_agent_simple = current_agent_simple[0].to(self.device)
        current_agent = current_agent[0].to(self.device)
        current_phase = current_phase[0].to(self.device)
        current_troops_count = current_troops_count[0].to(self.device)

        info = dict({})


        
        hero_steps = current_agent_simple == self.hero

        states = torch.cat((states, action_masks * hero_steps,
                              current_phase, current_agent,
                              current_troops_count[:, :, None]), axis=2)  # ,torch.ones(len(action_masks))[:,None]*self.hero



        #print(actions_1.requires_grad,actions_2.requires_grad)

        (action_pred_1,action_pred_2) =  (actions_1* hero_steps[:,0],
                                                        actions_2* hero_steps[:,0]
                                                         )

        # state_preds, action_preds, return_preds = self.model.forward(
        #                                                timesteps=timesteps,
        #                                                states=states,
        #                                                actions=actions,
        #                                                returns_to_go=returns_to_go
        #                                                #,info = info
        #                                            )

        #so this return to go is for value loss estimation

        _,_, returntogo_pred_v,_ = self.model.forward(
                                                        timesteps=timesteps,
                                                        states=states,
                                                        actions_1=action_pred_1,actions_2=action_pred_2,
                                                        returns_to_go=returntogo,
                                                            print_=print_
                                                        #,info = info
                                                    )
        
        # this is also for value function loss estimation

        (#state_preds_target, 
         _,_,return_preds_target,_) =  self.target_model.forward(timesteps=timesteps,
                                                                                states=states, actions_1=action_pred_1,actions_2=action_pred_2,
                                                                                returns_to_go=returntogo, print_=print_)


        
        
        (#state_preds_target, 
         return_preds_target) = (return_preds_target.detach())

        #if (not torch.isfinite(return_preds_target[:, -1]).all()
        #   ) or (not torch.isfinite(returntogo_pred_v[:, -1]).all()
        #        ) or (not torch.isfinite(action_pred_1[:, -1]).all()
        #             ) or (not torch.isfinite(action_pred_2[:, -1]).all()
        #                  )     :
        #    
        #    #print(action_model_2_dir[:, -1,0])
        #    
        #    print('returntogo_pred_v',returntogo_pred_v[:, -1])
        #    print('return_preds_target',return_preds_target[:, -1])
        #    
        #    print('action_pred_2',action_pred_2[:, -1])
        #    print('action_pred_1',action_pred_1[:, -1].long())
        #    a()

        #if True:
        #    print('traj_mask',traj_mask.requires_grad)
        #    print('actions',actions.requires_grad)
        #    print('returntogo_pred',returntogo_pred.requires_grad)
        #    print('reward', reward.requires_grad)
        #    print('returntogo',returntogo.requires_grad)
        #    print('returns_to_go_cal',returns_to_go_cal.requires_grad)
        #    print('hero_steps',hero_steps.requires_grad)
        #    a()
        if  len(reward[:, -1].squeeze().view(-1).shape) == 0:
            
            print('return_preds_last',return_preds_target.shape)
            print('reward_last',reward.shape)
            print('returns_to_go_cal_last',returns_to_go_cal.shape)

            print('return_preds_last',return_preds_target)
            print('reward_last',reward)
            print('returns_to_go_cal_last',returns_to_go_cal)




        
        total_loss, (Q_TD, Q_MSE, Q_loss, policy_loss) = self.calculate_loss(
                                                                                timesteps,
                                                                                return_preds_target,#target model predictions 
                                                                                returns_to_go_cal, # Q calculated
                                                                                hero_steps    ,#current_agent_acting,
                                                                                states,
                                                                                actions_1,actions_2,log_probs_actions_2,
                                                                                returntogo_pred,
                                                                                returntogo_pred_v,
                                                                                reward,#actual rewards 
                                                                                returntogo,  #actual R2G given to the model
                                                                                action_masks,
                                                                                print_,
                                                                                chunk_id,divi
                                                                                )
        #if chunk_id == 0:
        #    print (chunk_id, total_loss)

        # action_loss = #nn.CrossEntropyLoss().forward(action_preds_2,action_target)#F.mse_loss(action_preds_2, action_target.float(), reduction='mean')

        self.optimizer_1.zero_grad()
        total_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), 0.25)
        self.optimizer_1.step()
        #self.scheduler.step()
        
        





        
        

        # policy_loss.backward(retain_graph=True)

        return total_loss.detach().item() ,Q_TD.detach().item(), Q_MSE.detach().item(), Q_loss.detach().item(), policy_loss.detach().item()



    
        # if value_cal_loss:
        #    q_los = Q_loss.detach().cpu().item()
        #    writer.add_scalar("Q_loss", q_los, iyr+i_train_iter*num_updates_per_iter)
        #    log_Q_losses.append(q_los)
        #
        # pol_los = policy_loss.detach().cpu().item()
        # writer.add_scalar("Policy_loss", pol_los, iyr+i_train_iter*num_updates_per_iter)
        # log_action_losses.append(pol_los)

        # # evaluate on env
        # results = evaluate_on_env(model, device, context_len, env, rtg_target, rtg_scale,
        #                        num_eval_ep, max_eval_ep_len, state_mean, state_std,
        # ........................)
        #
        # writer.add_scalar("eval/avg_reward", results['eval/avg_reward'], i_train_iter)
        # writer.add_scalar("eval/avg_ep_len", results['eval/avg_ep_len'], i_train_iter)
        # writer.add_scalar("eval/'total_reward",results['total_reward'],i_train_iter)
        #
        #
        #
        # eval_avg_reward = results['eval/avg_reward']
        # eval_avg_ep_len = results['eval/avg_ep_len']
        # eval_d4rl_score = results['eval/avg_reward']#get_d4rl_normalized_score(results['eval/avg_reward'], env_name) * 100
        # mean_action_loss,mean_Q_loss = np.mean(log_action_losses),np.mean(log_Q_losses)
        #
        # writer.add_scalar("mean_action_loss_per_iter", mean_action_loss, i_train_iter)
        # writer.add_scalar("mean_Q_loss_per_iter", mean_Q_loss, i_train_iter)
        #
        #
        #
        # time_elapsed = str(datetime.now().replace(microsecond=0) - start_time)
        # total_updates += num_updates_per_iter
        # log_str = ("=" * 60 + '\n' +
        # ........"time elapsed: " + time_elapsed  + '\n' +
        # ........"num of updates: " + str(total_updates) + '\n' +
        # ........"action loss: " +  format(mean_action_loss, ".5f") + '\n' +
        # ........"Q loss: " +  format(mean_Q_loss, ".5f") + '\n' +
        # ........"eval avg reward: " + format(eval_avg_reward, ".5f") + '\n' +
        # ........"eval avg ep len: " + format(eval_avg_ep_len, ".5f") + '\n' +
        # ........"eval d4rl score: " + format(eval_d4rl_score, ".5f") +'\n'+
        #        " iterations: "+str(i_train_iter) +" calulate_value_loss :" +str(value_cal_loss)
        # ........)
        #
        # print(log_str)
        #
        # log_data = [time_elapsed, total_updates, mean_action_loss,mean_Q_loss,
        # ............eval_avg_reward, eval_avg_ep_len,
        # ............eval_d4rl_score]
        #
        # csv_writer.writerow(log_data)
        #
        # # save model
        # print("max d4rl score: " + format(max_d4rl_score, ".5f"))
        # if eval_d4rl_score >= max_d4rl_score:
        #    print("saving max d4rl score model at: " + save_best_model_path)
        #    torch.save(model.state_dict(), save_best_model_path)
        #    max_d4rl_score = eval_d4rl_score
        #
        # print("saving current model at: " + save_model_path)
        # torch.save(model.state_dict(), save_model_path)

        # writer.add_hparams(dict(exp_name=log_csv_name,env_name=env_name,state_dim=state_dim,
        # ............act_dim=int(act_dim),
        # ............n_blocks=n_blocks,
        # ............h_dim=embed_dim,
        # ............context_len=context_len,
        # ............n_heads=n_heads,
        # ............drop_p=dropout_p,training_division=training_division,
        #            start_value_training_iteration=start_value_training_iteration,optim=optimizer_1.__class__.__name__,lr=lr,
        #                       scheduler=scheduler.__class__.__name__, sch_param = ",".join([ i+":"+str(j) for i,j in dict(T_0=150, T_mult=2, eta_min=0.01, last_epoch=-1).items()])
        #                       ,batch_size=batch_size),{'hparam/eval_avg_reward':eval_avg_reward})

        # print("=" * 60)
        # print("finished training!")
        # print("=" * 60)
        # end_time = datetime.now().replace(microsecond=0)
        # time_elapsed = str(end_time - start_time)
        # end_time_str = end_time.strftime("%y-%m-%d-%H-%M-%S")
        # print("started training at: " + start_time_str)
        # print("finished training at: " + end_time_str)
        # print("total training time: " + time_elapsed)
        # print("max d4rl score: " + format(max_d4rl_score, ".5f"))
        # print("saved max d4rl score model at: " + save_best_model_path)
        # print("saved last updated model at: " + save_model_path)
        # print("=" * 60)

    def init_path(self):
        self.paths = []

    def init_CL_sample_store(self):
        self.DT_input = {  # self.actor_config_dict['ob_space']
            'timestep': torch.zeros((1,
                                    self.context_len),requires_grad =False).to(self.device,
                    dtype=torch.int),
            'state': torch.zeros((1, self.context_len,
                                 self.state_dim),requires_grad =False).to(self.device,dtype=torch.float32),
            'action_1': torch.zeros((1, self.context_len
                                  )).to(self.device,dtype=torch.float32),
            'action_2': torch.zeros((1, self.context_len
                                  )).to(self.device,dtype=torch.float32),
            
            'return_to_go': torch.ones((1,
                    self.context_len),requires_grad =False).to(self.device,dtype=torch.float32) * 110,
            }

        self.returntogo = torch.zeros((self.num_steps,
                1),requires_grad =False).to(self.device,dtype=torch.float32)  # self.total_agents
        self.returntogo_pred = torch.zeros((self.num_steps,
                1)).to(self.device,dtype=torch.float32)  # self.total_agents

    #here
    def current_model_in(
        self,
        observation,
        curr_agent,
        phase_mapping,
        curr_agent_mapping,
        env_board_agents=[],
        ):




        self.model_in =  torch.hstack((observation['observation'
                         ].reshape(-1).to(self.device),
                         torch.tensor(observation['action_mask'
                         ].reshape(-1)).to(self.device) * (curr_agent
                         == self.hero), phase_mapping.to(self.device),
                         curr_agent_mapping.to(self.device),
                         ( (torch.tensor([env_board_agents[self.hero].bucket]).to(self.device) - 5.2496)/1.4733
                                      )))[None,
                         :].float().requires_grad_(False).to(self.device)  




    

    def update_CL_sample_store(
        self,
        curr_agent_,
        inp={'step': None, 'act_2_1': [],'act_2_2': [], 'curr_reward_list': []},
        before_action=True,
        ):

        if before_action == 1 :

            if inp['step'] == 0:

                # print(self.model_in.shape)
                # print(self.model_in.repeat(self.context_len).shape)

                self.DT_input['state'] = self.model_in.repeat(self.context_len,
                        1).to(self.device)[None, :]
            else:

            # if step<self.context_len:

            #    trace[step] = model_in

                self.DT_input['state'][0, 0:-1] = self.DT_input['state'
                        ][0, 1:].clone()
                self.DT_input['state'][0, -1] = self.model_in
                self.DT_input['timestep'][0, 0:-1] = self.DT_input['timestep'][0, 1:].clone()
                self.DT_input['timestep'][0, -1] = inp['step']
                self.DT_input['action_1'][0, 0:-1] = self.DT_input['action_1'][0, 1:].clone()
                self.DT_input['action_2'][0, 0:-1] = self.DT_input['action_2'][0, 1:].clone()
                
                self.DT_input['return_to_go'][0, 0:-1] = self.DT_input['return_to_go'][0, 1:].clone()
        elif before_action == 2 :
            if self.hero == curr_agent_:
                self.DT_input['action_1'][0, -1] = inp['act_2_1']
            else:
                self.DT_input['action_1'][0, -1] = 0

        elif before_action == 3  :
            if self.hero == curr_agent_:
                self.DT_input['action_2'][0, -1] = inp['act_2_2']
            else:
                self.DT_input['action_2'][0, -1] = 0
        else:
                
            self.DT_input['return_to_go'][0, -1] = self.DT_input['return_to_go'][0, -1] -    inp['curr_reward_list']  # [self.hero]
            self.returntogo[inp['step']] = self.DT_input['return_to_go'
                    ][0, -1]

    def update_train_data(
        self,
        step_count,
        obs,
        ob_space_shape,
        rewards_2,
        dones_2,
        actions_1,
        actions_2,
        log_probs_actions_2,
        action_masks,
        current_agent,
        current_agent_acting,
        current_phase,
        current_troops_count,
        map_agent_phase_vector,
        ):

        
        
        data_ = collections.defaultdict(torch.tensor)
        data_['observations'] =             obs[:step_count].reshape(-1,
                np.prod(ob_space_shape))

                # data_['next_observations'] = obs[1:step_count+1].to(device =self.args.pin_memory_device).reshape(-1,np.prod(T.ob_space_shape)) #torch.tensor([1,2,3,4])
                # this return to go is the actual input

        data_['returntogo'] =             self.returntogo[:step_count]  # torch.tensor([1,2,3,4])
        data_['returntogo_pred'] =             self.returntogo_pred[:step_count]  # torch.tensor([1,2,3,4])

        data_['rewards'] =             rewards_2[:step_count]  # torch.tensor([1,2,3,4])
        data_['terminals'] =             dones_2[:step_count]  # torch.tensor([1,2,3,4])
        data_['actions_1'] =             actions_1[:step_count]  # torch.tensor([1,2,3,4])
        data_['actions_2'] =             actions_2[:step_count]  # torch.tensor([1,2,3,4])
        data_['log_probs_actions_2'] =   log_probs_actions_2[:step_count]
        data_['action_masks'] =             action_masks[:step_count]
        data_['current_agent_acting'] =     current_agent_acting[:step_count]
        data_['current_agent_simple'] =     current_agent[:step_count]
        data_['current_agent'] =             map_agent_phase_vector(current_agent[:step_count],
                                   num_classes=self.total_agents + 1)[:
                , 1:]
        data_['current_phase'] =             map_agent_phase_vector(current_phase[:step_count],
                                   num_classes=self.total_phases)
        data_['current_troops_count'] =             current_troops_count[:step_count]



        
        
        self.datase = TrajectoryDataset_per_episode([data_], #len(data[0]['observations'])<self.args.context_len
                context_len=self.args.context_len,rtg_scale=self.args.rtg_scale,dev = self.args.pin_memory_device,
                 gamma=self.args.gamma)

        self.path_que(DataLoader(self.datase, batch_size=len(self.datase)))
    def path_que(self, dtl):
        
        if (len(self.paths)==self.rb_len):
            if (self.rb_len >1):
                self.paths.pop(random.randrange(len(self.paths)-self.args.num_episodes +1 )) # dont pop the most recent experiences and ensure rb_len > num.episodes
            else: 
                self.paths.pop()
        self.paths.append(dtl)
        
    def create_training_dataset(self):
        self.traj_dataset =  TrajectoryDataset_2_through_episodes(self.paths)  # a dataset of dataloaders

        self.traj_data_loader = DataLoader(  # only spit 1 episode a time
            self.traj_dataset,
            batch_size=1,
            shuffle=self.args.shuffle,
            pin_memory=self.args.pin_memory,
            drop_last=self.args.drop_last,
            pin_memory_device=self.args.pin_memory_device,
            )

    def action_predict(self, save_R=True, return_R=False,shift=1,action_masks = [],return_log_prob_a2 = False): 
        (#s, 
         a_1,a_2, R,lp) = self.model(timesteps=self.DT_input['timestep'],
                               states=self.DT_input['state'],
                               actions_1=self.DT_input['action_1'],
                               actions_2=self.DT_input['action_2'],
                               returns_to_go=self.DT_input['return_to_go'
                               ][:, :, None], 
                                return_log_prob_a2 = return_log_prob_a2)


        took_action = False
        
        if len(action_masks) > 0:
            #handling when all the probability of masked actions are zero.... #have to force the model to pick 1st valid action.
            masked_action = (a_1[0, -1, :]*action_masks)
            #valid_ind = torch.nonzero(action_masks).squeeze()
            if torch.any(masked_action !=0):
                took_action = True
                
                action_1 = (masked_action).argmax()[None]
            else: #im hoping that slowly and steadily the model would learn to predict non zero probabilities for action mask
                
                action_1 = torch.tensor(np.random.choice(torch.where(action_masks)[0]))[None]  #(masked_action + 0.0000001*action_masks).argmax()[None]
        else:
            action_1 = a_1[0, -1, :].argmax()[None]
            
        action_2 = a_2[0,-1, 0][None]
        if return_log_prob_a2:
            log_prob = lp[0,-1, 0][None]
        else:
            log_prob = None
        
        if save_R:
            self.returntogo_pred[self.DT_input['timestep'][0, -1]] =                 R[0, -1]  # R

        if return_R:
            return (action_1, action_2, R[0, -1],took_action,log_prob)
        else:
            return action_1, action_2,took_action,log_prob

    def action_predict_direct(self, data, return_R=False,return_log_prob_a2 = False):
        (timesteps, states, actions_1,actions_2, returns_to_go) = data
        (#s, 
         a_1,a_2, R,lp) = self.model(timesteps=timesteps, states=states,
                               actions_1=actions_1,actions_2=actions_2,
                               returns_to_go=returns_to_go,return_log_prob_a2 = return_log_prob_a2)
        if return_R:
            return (a_1,a_2, R,lp)
        else:
            return a_1,a_2,lp

    def save_models(self):
        newpath = r'./models/' + self.run_name + '/' + str(self.hero)
        if not os.path.exists(newpath):
            os.makedirs(newpath)
        torch.save(self.model.state_dict(), newpath
                   + '/a2c_transformer.pt')

    def load_models(self):
        newpath = r'./models/' + self.run_name + '/' + str(self.hero)
        self.model.load_state_dict(torch.load(newpath
                                   + '/a2c_transformer.pt'))
        self.target_model.load_state_dict(torch.load(newpath
                + '/a2c_transformer.pt'))


## model selector

In [8]:
def model_selector(model_name="DDQN_module", kwarg = {}):
    model_list = {"DDQN_module":DDQN_module,"transformer_model":transformer_model}
    print(type(model_list[model_name]))
    return model_list[model_name](**kwarg)

# hero agent definition

In [9]:
import numpy as np
class Hero_agent(int):
    def init_properties(self,agent_count,phases,cp=[],df=[],direct_action=True):
        #self.draw_count = 0
        self.init_win_count_iter(agent_count)
        self.init_move_count_epi(phases)
        self.cp = cp
        self.df = df
        self.direct_action = direct_action
        self.init_reward_concern(agent_count,cp=cp,df=df)
        
    def init_reward_concern(self,agent_count,cp=[],df=[]):
        if len(cp)==0:
            cp = [int(self)]
        self.concern=torch.tensor([(1 if i in cp 
                             else 
                             (-1 if i in df 
                                  else 0)) for i in range(1,agent_count+1) ])
        #self.concern_2 = self.concern
        #self.concern_2[self-1] =0
        
        self.multi_dependency = (sum(self.concern !=0)>1)
        
        
    def init_model(self,model_name="DDQN_module",
                   kwarg = dict({})):
        self.model = model_selector(model_name=model_name, 
                                    kwarg = kwarg)

        
    def init_win_count_iter(self,agent_count):
        self.count_dict = {i:0 for i in range(1,agent_count+1)}
        self.count_draw_dict = {i:0 for i in range(1,agent_count+1)}
        self.draw_territory_count = 0
    def init_move_count_epi(self,phases):
        self.bad_move_count = 0
        self.bad_move_phase_count = {i:0 for i in phases}
        self.move_count =  {i:0 for i in phases}        
    
    def model_def(self, model):
        self.model =model

    def action_predict(self,save_R=True,return_R = False,action_masks = [],return_log_prob_a2 = False):
        return self.model.action_predict(save_R=save_R,return_R = return_R, action_masks = action_masks,return_log_prob_a2 = return_log_prob_a2)

    def action_predict_direct(self,data,return_R = False,return_log_prob_a2 = False):
        return self.model.action_predict_direct(data,return_R = return_R,return_log_prob_a2 = return_log_prob_a2)
    def save_models(self):
        self.model.save_models()

    def process_reward(self,rewards,step,hero_steps):
        if self.multi_dependency and self.direct_action:
            return (rewards*self.concern.to(rewards.device)).sum(-1)[:step][hero_steps][:,None]
        elif self.multi_dependency and not self.direct_action:
            base_rew = torch.zeros( rewards[:step,self-1][hero_steps].shape,require_grad=False)
            #print(base_rew)

            
            hero_step_list  = np.arange(0,step)[hero_steps]
            for i,j in zip(hero_step_list[:-1],hero_step_list[1:]):
                if j-i>1:
                    #print(j,i,rewards[i:j],(rewards[i:j]*self.concern),(rewards[i:j]*self.concern).sum())
                    base_rew[i]+= (rewards[i:j]*self.concern).sum()
            #print(base_rew,rewards[hero_step_list[-1]:],(rewards[hero_step_list[-1]:]*self.concern))
            base_rew[-1]+= (rewards[hero_step_list[-1]:]*self.concern).sum()
            
            return base_rew[:,None]
            
        else:
            return rewards[:step][hero_steps][:,None]
    
    #def model_forward_call(self,name,kwarg):
    #    return self.model_dict[name](**kwarg)
        

a = Hero_agent(1)
a.init_properties(3,[1,2,3],cp=[1],df=[2])

# dataset definition 
## episode trajectory
## dataloader for per iterations

In [10]:
#obs[:,-1] = (obs[:,-1]  - 5.2496)/1.4733

#        current_troops_count_mean tensor(5.2496), current_troops_count_std = tensor(1.4733)

#        (x-mean)/std

In [11]:
import torch
from torch.utils.data import Dataset
from torch.nn.functional import pad

class TrajectoryDataset_per_episode(Dataset): #this should have only 1 trajectory no matter what
    def __init__(self, trajectories, context_len, rtg_scale,dev,gamma=0.99,min_len = 10**6):
        self.trajectories = trajectories
        self.context_len = context_len
        self.dev = dev
        min_len = min(min_len, len(self.trajectories[0]['observations'])) ##len(data[0]['observations'])<self.args.context_len


        
        #states = []
        #for traj in self.trajectories:
        #    traj_len = traj['observations'].shape[0]
        #    min_len = min(min_len, traj_len)
        #    states.append(traj['observations'])
        #    # calculate returns to go and rescale them 

        self.pad_init()
        
        self.trajectories[0]['returns_to_go_cal'] = discount_cumsum(self.trajectories[0]['rewards'], gamma) / rtg_scale

        self.trajectories[0]['current_troops_count'] = (self.trajectories[0]['current_troops_count'] - 5.2496)/1.4733
        
        #print(min_len)
        
        # used for input normalization
        
        #states = torch.concatenate(states, axis=0).to(dtype = torch.float32)
        #self.state_mean, self.state_std = torch.mean(self.trajectories[0]['observations'], axis=0
        #                                            ), torch.std(self.trajectories[0]['observations'], axis=0) + 1e-6

        # normalize states
        #for traj in self.trajectories:

            #self.trajectories[0]['current_troops_count']
            #traj['observations'] = (traj['observations'].to(dtype=torch.float32) - self.state_mean) / self.state_std




        


    


    def pad_init(self):



        #observations : torch.Size([255, 40]) torch.Size([2442, 40]) 
        # returntogo : torch.Size([255, 1]) torch.Size([2442, 1]) 
        # returntogo_pred : torch.Size([255, 1]) torch.Size([2442, 1]) 
        # rewards : torch.Size([255]) torch.Size([2442]) 
        # terminals : torch.Size([255]) torch.Size([2442]) 
        # actions :
        #torch.Size([255, 2]) torch.Size([2442, 2]) 
        # action_masks : torch.Size([255, 32]) torch.Size([2442, 32]) 
        # current_agent_simple : torch.Size([255, 1]) torch.Size([2442, 1]) 
        # current_agent : torch.Size([255, 2]) torch.Size([2442, 2]) 
        # current_phase : torch.Size([255, 2]) torch.Size([2442, 2]) 
        # current_troops_count : torch.Size([255]) torch.Size([2442])


        


        self.trajectories[0]['observations'] =  torch.cat( (
                                              self.trajectories[0]['observations'][0].repeat([self.context_len -1]+[1 for i in range(len(self.trajectories[0]['observations'].shape)-1) ]),
                                              self.trajectories[0]['observations']
                                              
                                            ),dim=0)          
        

        self.trajectories[0]['returntogo'] =      torch.cat( (
                                              self.trajectories[0]['returntogo'][[0]].repeat([self.context_len -1]+[1 for i in range(len(self.trajectories[0]['returntogo'].shape)-1) ]),
                                              self.trajectories[0]['returntogo']
                                              
                                            ),dim=0)         # torch.tensor([1,2,3,4])
        self.trajectories[0]['returntogo_pred'] =             torch.cat( (
                                              self.trajectories[0]['returntogo_pred'][[0]].repeat([self.context_len -1]+[1 for i in range(len(self.trajectories[0]['returntogo_pred'].shape)-1) ]),
                                              self.trajectories[0]['returntogo_pred']
                                              
                                            ),dim=0)   # torch.tensor([1,2,3,4])

        self.trajectories[0]['rewards'] =    torch.cat( (
                                              torch.zeros([self.context_len -1]).to(device=self.dev),
                                              self.trajectories[0]['rewards']
                                              
                                            ),dim=0)  # torch.tensor([1,2,3,4])
        self.trajectories[0]['terminals'] =    torch.cat( (
                                              self.trajectories[0]['terminals'][[0]].repeat([self.context_len -1]+[1 for i in range(len(self.trajectories[0]['terminals'].shape)-1) ]),
                                              self.trajectories[0]['terminals']
                                              
                                            ),dim=0)  # torch.tensor([1,2,3,4])

        
        self.trajectories[0]['actions_1'] =      torch.cat( (
                                              torch.zeros([self.context_len -1]+list(self.trajectories[0]['actions_1'].shape[1:])).to(device=self.dev),
                                              self.trajectories[0]['actions_1']
                                              
                                            ),dim=0)
        
        self.trajectories[0]['actions_2'] =      torch.cat( (
                                              torch.zeros([self.context_len -1]+list(self.trajectories[0]['actions_2'].shape[1:])).to(device=self.dev),
                                              self.trajectories[0]['actions_2']
                                              
                                            ),dim=0)

        self.trajectories[0]['log_probs_actions_2'] = torch.cat( (
                                              torch.zeros([self.context_len -1]+list(self.trajectories[0]['log_probs_actions_2'].shape[1:])).to(device=self.dev),
                                              self.trajectories[0]['log_probs_actions_2']
                                              
                                            ),dim=0)
        
        
        
        
        # torch.tensor([1,2,3,4])
        self.trajectories[0]['action_masks'] =    torch.cat( (
                                              torch.zeros([self.context_len -1]+list(self.trajectories[0]['action_masks'].shape[1:])).to(device=self.dev,dtype=torch.float32),
                                              self.trajectories[0]['action_masks']
                                              
                                            ),dim=0)



        self.trajectories[0]['current_agent_acting'] =     torch.cat( (
                                                              torch.zeros([self.context_len -1]+list(self.trajectories[0]['current_agent_acting'].shape[1:])).to(device=self.dev,dtype=torch.float32),
                                                              self.trajectories[0]['current_agent_acting']
                                                              
                                                            ),dim=0)
        
        self.trajectories[0]['current_agent_simple'] =     torch.cat( (
                                                              torch.zeros([self.context_len -1]+list(self.trajectories[0]['current_agent_simple'].shape[1:])).to(device=self.dev,dtype=torch.float32),
                                                              self.trajectories[0]['current_agent_simple']
                                                              
                                                            ),dim=0)

        
        self.trajectories[0]['current_agent'] =             torch.cat( (
                                                              torch.zeros([self.context_len -1]+list(self.trajectories[0]['current_agent'].shape[1:])).to(device=self.dev,dtype=torch.float32),
                                                              self.trajectories[0]['current_agent']
                                                              
                                                            ),dim=0)

        
        self.trajectories[0]['current_phase'] =             torch.cat( (
                                                          self.trajectories[0]['current_phase'][0].repeat([self.context_len -1]+[1 for i in range(len(self.trajectories[0]['current_phase'].shape)-1) ]),
                                                          self.trajectories[0]['current_phase']
                                                          
                                                        ),dim=0) 
        self.trajectories[0]['current_troops_count'] =      torch.cat( (
                                              self.trajectories[0]['current_troops_count'][0].repeat([self.context_len -1]),
                                              self.trajectories[0]['current_troops_count']
                                              
                                            ),dim=0)




    
    

    def __len__(self):
        #print(len(self.trajectories),(self.trajectories[0].shape),len(self.trajectories[0])- self.context_len + 1 )
        return sum(max(0, len(traj['observations'])- self.context_len + 1
                      ) for traj in self.trajectories)

    def __getitem__(self, idx):
        total_len = 0
        for traj in self.trajectories:
            
            #print(total_len, idx - total_len, total_len + len(traj), - self.context_len + 1)
            
            if total_len  <= idx < total_len + len(traj['observations']) - self.context_len + 1    :
                si = idx - total_len
                
                #context = traj[si:si + self.context_length]
                states = (traj['observations'][si : si + self.context_len])
                
                actions_1 = traj['actions_1'][si : si + self.context_len]#torch.cat((traj['actions_1'][si : si + self.context_len-1].clone().detach(),   traj['actions_1'][[si + self.context_len-1]]      ),dim =0)
                actions_2 = traj['actions_2'][si : si + self.context_len]#torch.cat((traj['actions_2'][si : si + self.context_len-1].clone().detach(),   traj['actions_2'][[si + self.context_len-1]]      ),dim =0)

                log_probs_actions_2 = traj['log_probs_actions_2'][si : si + self.context_len]
                
                reward =  (traj['rewards'][si : si + self.context_len])
                returntogo = (traj['returntogo'][si : si + self.context_len])
                returns_to_go_cal = (traj['returns_to_go_cal'][si : si + self.context_len])
                returntogo_pred = torch.cat((traj['returntogo_pred'][si : si + self.context_len-1].clone().detach(),  traj['returntogo_pred'][[ si + self.context_len-1]]       ),dim =0)




                
                action_masks = (traj['action_masks'][si : si + self.context_len])

                current_agent_acting = (traj['current_agent_acting'][si : si + self.context_len])
                current_agent = (traj['current_agent'][si : si + self.context_len])
                current_agent_simple = (traj['current_agent_simple'][si : si + self.context_len])
                current_phase = (traj['current_phase'][si : si + self.context_len])
                current_troops_count = (traj['current_troops_count'][si : si + self.context_len])
    
                
                timesteps = torch.arange(start=si, end=si+self.context_len, step=1)
    
                # all ones since no padding
                traj_mask = torch.ones(self.context_len, dtype=torch.long)



                if self.context_len> len(states):
                    padding_len = self.context_len - len(states)
    
                    states                = torch.cat([states,
                                    torch.zeros(([padding_len] + list(states.shape[1:])),
                                    dtype=reward.dtype
                                               )], 
                                   dim=0)  
                    actions_1               = torch.cat([actions_1,
                                    torch.zeros(([padding_len] + list(actions_1.shape[1:])),
                                    dtype=reward.dtype
                                               )], 
                                   dim=0)
                    
                    actions_2               = torch.cat([actions_2,
                                    torch.zeros(([padding_len] + list(actions_2.shape[1:])),
                                    dtype=reward.dtype
                                               )], 
                                   dim=0)

                    log_probs_actions_2 =  torch.cat([log_probs_actions_2,
                                    torch.zeros(([padding_len] + list(log_probs_actions_2.shape[1:])),
                                    dtype=reward.dtype
                                               )], 
                                   dim=0)
                    
                    reward                = torch.cat([reward,
                                    torch.zeros(([padding_len] + list(reward.shape[1:])),
                                    dtype=reward.dtype
                                               )], 
                                   dim=0)  


                    returntogo            = torch.cat([returntogo,
                                    torch.zeros(([padding_len] + list(returntogo.shape[1:])),
                                    dtype=reward.dtype
                                               )], 
                                   dim=0)
                    returns_to_go_cal     = torch.cat([returns_to_go_cal,
                                    torch.zeros(([padding_len] + list(returns_to_go_cal.shape[1:])),
                                    dtype=reward.dtype
                                               )], 
                                   dim=0)
                    returntogo_pred       = torch.cat([returntogo_pred,
                                    torch.zeros(([padding_len] + list(returntogo_pred.shape[1:])),
                                    dtype=reward.dtype
                                               )], 
                                   dim=0)



                    
                    action_masks          = torch.cat([action_masks,
                                    torch.zeros(([padding_len] + list(action_masks.shape[1:])),
                                    dtype=reward.dtype
                                               )], 
                                   dim=0)        

                    current_agent_acting = torch.cat([current_agent_acting,
                                    torch.zeros(([padding_len] + list(current_agent_acting.shape[1:])),
                                    dtype=reward.dtype
                                               )], 
                                   dim=0)
                    
                    current_agent         = torch.cat([current_agent,
                                    torch.zeros(([padding_len] + list(current_agent.shape[1:])),
                                    dtype=reward.dtype
                                               )], 
                                   dim=0)
                    
                    current_troops_count = torch.cat([current_troops_count,
                                        torch.zeros(([padding_len] + list(current_troops_count.shape[1:])),
                                        dtype=current_troops_count.dtype
                                                   )], 
                                       dim=0)
                    current_phase         = torch.cat([current_phase,
                                    torch.zeros(([padding_len] + list(current_phase.shape[1:])),
                                    dtype=reward.dtype
                                               )], 
                                   dim=0)         
                    current_troops_count  = torch.cat([current_troops_count,
                                    torch.zeros(([padding_len] + list(current_troops_count.shape[1:])),
                                    dtype=reward.dtype
                                               )], 
                                   dim=0)                
                    traj_mask             = torch.cat([traj_mask,
                                    torch.zeros(([padding_len] + list(traj_mask.shape[1:])),
                                    dtype=reward.dtype
                                               )], 
                                   dim=0)     














                
                return  timesteps, states, actions_1,actions_2,log_probs_actions_2, returntogo, returns_to_go_cal, returntogo_pred,reward, traj_mask ,action_masks,current_agent_acting,current_agent_simple,current_agent,current_phase,current_troops_count
                
                #return pad(torch.tensor(context), (0,(self.context_length - len(context))),mode="constant"), [1]

            total_len += len(traj) - self.context_len + 1

        raise IndexError("Index out of range 1")


class TrajectoryDataset_2_through_episodes(Dataset):
    def __init__(self, trajectories):
        self.trajectories = trajectories

        #all_obs = torch.concat([ traj.dataset.trajectories[0]['observations'] for traj in self.trajectories],axis=0)
        #self.state_mean = torch.mean(all_obs,axis =0)
        #self.state_std = torch.std(all_obs,axis =0) + 1e-6
        
        #for traj in self.trajectories:
        #    traj.dataset.trajectories[0]['observations'] = (traj.dataset.trajectories[0]['observations'].to(dtype=torch.float32) - self.state_mean) / self.state_std


        #print(self.state_mean,self.state_std)
        
    #def get_state_stats(self):
        #return self.state_mean, self.state_std        

    

    def __len__(self):
        return len(self.trajectories)

    def __getitem__(self, idx):
        total_len = 0
        if total_len  <= idx < total_len + len(self.trajectories)  :
            return [batch for batch in self.trajectories[idx] ][0]


        raise IndexError("Index out of range 2")

# trainer module

In [12]:








class Trainer:
    """
    ## Trainer
    """

    def __init__(self, Args,param_dict =dict({})
                 ):
        # #### Configurations

        self.args = Args()#tyro.cli(Args)
        self.param_dict = param_dict
        self.update_arg(param_dict=param_dict)
        self.device = self.args.device#torch.device("cuda" if torch.cuda.is_available() and self.args.cuda else "cpu")
        
        
        #self.args.batch_size = int(self.args.num_envs * self.args.num_steps)
        self.args.minibatch_size = int(self.args.batch_size // self.args.num_minibatches)
        #self.args.num_iterations = self.args.total_timesteps // self.args.batch_size
        self.gam = self.args.gamma
        #self.args.minibatch_size = 256#128 
        self.num_steps = self.args.num_steps#120000#1000000
        self.num_iterations = self.args.num_iterations
        self.episode_time_lim = self.args.episode_time_lim
        self.hero_agent_count = self.args.hero_agent_count
        self.env_config = self.args.env_config
        self.num_episodes = self.args.num_episodes
        self.context_len=self.args.model_config['context_len'] #200

        self.training_performance_return = []
        
        #self.env_config = dict(render_mode = 'rgb_array', default_attack_all  = True,
        #                    agent_count  = 4
        #                       ,use_placement_perc=True,render_=False)        
        
        self.run_name = f"{self.args.env_id}__{self.args.exp_name}__{self.args.seed}__{int(time.time())}"

        

        TB_log = self.args.TB_log 
        if TB_log:    
            self.writer = SummaryWriter(f"runs/{self.run_name}")
            self.writer.add_text(
                "hyperparameters",
                "|param|value|\n|-|-|\n%s" % ("\n".join([f"|{key}|{value}|" for key, value in vars(self.args).items()])),
            )
        else:
            self.writer = None
        
        # TRY NOT TO MODIFY: seeding
        random.seed(self.args.seed)
        np.random.seed(self.args.seed)
        #torch.manual_seed(self.args.seed)
        
        torch.backends.cudnn.deterministic = self.args.torch_deterministic
        
        
        
        self.playe_r = 1#"agent_1" #
        

        
        self.action_shape = (2,)



        self.env = env_risk(**self.env_config)
        
        self.env.reset(seed=42)


            

        
        self.args.total_agents = self.total_agents  = len(self.env.possible_agents)
        self.args.total_phases = self.total_phases = len(self.env.phases)

        
        

        
        print(torch.tensor(self.env.last()[0]['observation']
                          ))

        print(torch.tensor(self.env.last()[0]['observation']
                          ).to(device=self.device))
        sample_obs = self.obs_converter(torch.tensor(self.env.last()[0]['observation']
                                                    ).to(device=self.device),
                                        num_classes = self.total_agents+1
                                       )
        
        self.ob_space_shape = sample_obs.shape #env.observation_space(playe_r)['observation'].shape
        self.action_mask_shape = self.env.observation_space(self.playe_r)['action_mask'].shape
        
        #self.device = torch.device("cuda" if torch.cuda.is_available() and args.cuda else "cpu")

        
        
        self.agent_list = list(self.env.possible_agents)
        
        
        
        
       
        self.the_hero_agent = 1

        
        self.qnet_config_dict = dict(action_space = self.env.action_space(self.playe_r
                                                                         ).shape[0],
                                    ob_space=(np.prod(self.ob_space_shape
                                                    )+np.prod(self.action_mask_shape)
                                         +1*( self.total_agents -1) #the current_agent +1#who actor agent was
                                         +1*(self.total_phases -1)#the current phase
                                         +1 )# the number of troops
                               )
        self.actor_config_dict =  dict(env=self.env,
                        action_space = self.env.observation_space(self.playe_r)['action_mask'].shape[0],
                        ob_space=(np.prod(self.ob_space_shape)
                                         +np.prod(self.action_mask_shape)
                                         +1*( self.total_agents-1) #the current_agent +1#who actor agent was
                                         +1*(self.total_phases -1)#the current phase
                                         +1) # the number of troops
                               )



        
        #torch.Tensor(torch.hstack((observation['observation'].reshape(-1),
        #                            torch.tensor(observation['action_mask'].reshape(-1)).to(self.device),
        #                                   phase_mapping,
        #                                    curr_agent_mapping,
        #                                   torch.tensor([env.board.agents[curr_agent].bucket ]).to(self.device)))[None,:]#.repeat(3,axis = 0)
        #                                        ).float()
                        
        
        self.hero_agents_list = {i:Hero_agent(i) for i in range(1,self.hero_agent_count+1) } # this is a list , need to pass it as an argument
        
        for i in self.hero_agents_list:
            self.hero_agents_list[i].init_properties(self.total_agents,self.env.phases)        


            print(len(self.args.model_name[i]))
            
            self.hero_agents_list[i].init_model(model_name=self.args.model_name[i], kwarg = dict(

                                                qnet_config_dict = self.qnet_config_dict, 
                                                actor_config_dict = self.actor_config_dict,
                                                args = self.args,
                                                device = self.device,
                                                writer=self.writer,
                                                run_name=self.run_name,
                                                agent=i)
                                                )
            

            #self.target_actor.load_state_dict(self.actor.state_dict())
            #self.qf1_target.load_state_dict(self.qf1.state_dict())
            #self.q_optimizer = optim.Adam(list(self.qf1.parameters()), lr=self.args.learning_rate)
            #self.actor_optimizer = optim.Adam(list(self.actor.parameters()), lr=self.args.learning_rate)

    def update_arg(self,param_dict=dict({})):
       for i,j in param_dict.items():
           setattr(self.args,i,j)

        

    def obs_converter(self,  data, num_classes = 4, col =0 ):

        if col != None:

            #print(data.device)
            #print(nn.functional.one_hot(data[:4,col].detach().long(), 
            #                                            num_classes = num_classes).to(self.device))
            return torch.concat((nn.functional.one_hot(data[:,col].detach().long(), 
                                                        num_classes = num_classes).to(self.device),
                                      data[:,~col,None]
                                ),axis=1
                               )[:,1:].to(self.device)
    
    def map_agent_phase_hot(self, data,num_classes = 3):
        with torch.no_grad():
            return nn.functional.one_hot(torch.tensor(data),num_classes = num_classes)[1:].to(self.device)
    
    def map_agent_phase_vector(self, data,num_classes = 3):
        with torch.no_grad():
            return nn.functional.one_hot(data[:,0].long(), 
                                                            num_classes = num_classes)[:,1:].to(self.device)


    def train_loop_init(self):
        self.gamma_t = {i:0 for i in self.env.possible_agents}
        
        
        self.draw_count = 0

        for i in self.hero_agents_list:
            self.hero_agents_list[i].init_win_count_iter(self.total_agents )
            self.hero_agents_list[i].model.init_path()
        
         
        #self.first_count = 0
        #self.second_count = 0
        #self.third_count = 0
        #self.third_count_draw = 0
        
        self.start_time = time.time()
        self.global_step = 0
        #self.faulting_player = ""

    
    
    def run_training_loop(self):
        """
        ### Run training loop
        """

        # last 100 episode information
        #tracker.set_queue('reward', 100, True)
        #tracker.set_queue('length', 100, True)


        for i in self.hero_agents_list: # each agent has his own buffer, this is kinda pain because now this information is stored and not discarded
    
            self.hero_agents_list[i].rb = ReplayBuffer(
                    self.args.buffer_size,
                    Box(low =0, high=2000, shape =(self.qnet_config_dict['ob_space']+1,), dtype=np.float32),
                    Box(low =0, high=2000, shape =(2,), dtype=np.float32),
                    self.device,
                    handle_timeout_termination=False,
                )

        env = env_risk(**(self.env_config #| {"render_mode" : None, "bad_mov_penalization" : 0.01,"render_":False}
                         ))
        env.reset(42)
        
        self.train_loop_init()
        #self.paths=[]
        
        self.training_performance_return = []
        
        for iteration in range(1, self.num_iterations+1):


            
            self.sample(
                                env,iteration,
                                
                                
                 
                            )

            for i in self.hero_agents_list:
                    self.hero_agents_list[i].model.create_training_dataset()

            

            #self.traj_dataset = D4RLTrajectoryDataset(self.paths, context_len=self.args.context_len,
            #                                             rtg_scale=self.args.rtg_scale,gamma=self.args.gamma)
            #                                            #rtg_scale=1,gamma=0.99
    
            #traj_dataset = traj_dataset.to(self.args.pin_memory_device)
            
            #self.traj_data_loader = DataLoader(self.traj_dataset,
            #						batch_size=self.args.batch_size,
            #                        shuffle= self.args.shuffle,
            #                        pin_memory= self.args.pin_memory,
            #                        drop_last = self.args.drop_last,
            #                        pin_memory_device=self.args.pin_memory_device
            #                        #shuffle=True,
            #                        #pin_memory=True,
            #                        #drop_last=True
            #                        )

            #self.traj_dataset = TrajectoryDataset_2_through_episodes(self.paths ) # a dataset of dataloaders
            #self.traj_data_loader = DataLoader(self.traj_dataset, batch_size=1,shuffle=self.args.shuffle,
            #                        pin_memory= self.args.pin_memory,
            #                        drop_last = self.args.drop_last,
            #                        pin_memory_device=self.args.pin_memory_device) # only spit 1 episode a time


            
            
            #self.data_iter = iter(self.traj_data_loader)
            
            ## get state stats from dataset
            #state_mean, state_std = self.traj_dataset.get_state_stats()

            #print(len(self.paths))
            #print(len(traj_dataset))
            
            #print(len(self.data_iter))
            #print(next(data_iter))
            #(timesteps, states, actions, returns_to_go,reward, traj_mask ,
            # action_masks,current_agent,current_phase,current_troops_count) = next(self.data_iter)

            #for batch in self.traj_data_loader

            #print(states.shape)
            
            




            
            self.train(iteration)
            #break

            
            
            #if self.global_step%100 ==0:
            #    SPS = int(self.global_step / (time.time() - self.start_time))
            #    print("SPS:", SPS)       
            #    self.writer.add_scalar("charts/SPS", SPS, self.global_step)
        
            
            self.save_models()

    def train(self,iteration):

        if True:#self.global_step > self.args.learning_starts:

            for i in self.hero_agents_list:
                self.hero_agents_list[i].model.train_write(
                        iteration,print_=True)
                
                        
                    
    
    def train_(self,iteration):
        
        for epoch in range(self.args.update_epochs):
            
            if self.global_step > self.args.learning_starts:

                for i in self.hero_agents_list:
                    self.hero_agents_list[i].model.train_write(self.hero_agents_list[i].rb.sample(self.args.batch_size)
                                                         ,iteration,epoch)
                    
    def save_models(self):
        for i in self.hero_agents_list:
            self.hero_agents_list[i].save_models()  

    def reset_moves_hero_agents(self):
        for i in self.hero_agents_list:
            self.hero_agents_list[i].init_move_count_epi(self.env.phases)
    
    def sample(self,env,iteration
                                ):
        

        #for i in self.hero_agents_list:
        #    self.hero_agents_list[i].model.init_path()
        
        with torch.no_grad():
        #if True:
            # sample `worker_steps` from each worker
            #there are no worker steps... rather there are full episodes

            step = 0
            fault_condition = False
            clear_output(wait=True)
            phase = 0
            
        



            

            
            for episode in range(self.num_episodes):#num_episodes):
                obs = torch.zeros((self.num_steps,) + self.ob_space_shape, requires_grad =False).to(self.device,    dtype = torch.float32)
                
                actions_1 = torch.zeros((self.num_steps,) ).to(self.device,    dtype = torch.float32)
                
                actions_2 = torch.zeros( (self.num_steps,)).to(self.device,    dtype = torch.float32)
                log_probs_actions_2 = torch.zeros( (self.num_steps,)).to(self.device,    dtype = torch.float32)
                
                action_masks = torch.zeros((self.num_steps, ) + self.action_mask_shape, requires_grad =False).to(self.device,    dtype = torch.float32)
                current_agent = torch.zeros((self.num_steps,1), requires_grad =False).to(self.device,    dtype = torch.float32)*0#-1
                current_agent_acting = torch.ones((self.num_steps,1), requires_grad =False).to(self.device,    dtype = torch.float32)*0
                current_phase = torch.zeros((self.num_steps,1), requires_grad =False).to(self.device,    dtype = torch.float32)
                current_troops_count = torch.zeros((self.num_steps,self.total_agents), requires_grad =False).to(self.device,    dtype = torch.float32)
                #logprobs = torch.zeros((self.num_steps, ), requires_grad =False).to(self.device,    dtype = torch.float32)
                rewards = torch.zeros((self.num_steps, self.total_agents), requires_grad =False).to(self.device,    dtype = torch.float32)
                rewards_2 = torch.zeros((self.num_steps, self.total_agents), requires_grad =False).to(self.device,    dtype = torch.float32)
                
                returntogo = torch.zeros((self.num_steps, self.total_agents), requires_grad =False).to(self.device,    dtype = torch.float32)
                dones = torch.zeros((self.num_steps, self.total_agents), requires_grad =False).to(self.device)
                dones_2 = torch.zeros((self.num_steps, self.total_agents), requires_grad =False).to(self.device)
                #values = torch.zeros((self.num_steps,  )).to(self.device)
                episodes = torch.ones((self.num_steps, ), requires_grad =False).to(self.device,    dtype = torch.float32)*-1
                t_next = torch.zeros((self.num_steps, self.total_agents), requires_grad =False).to(self.device,    dtype = torch.float32)
                    
                total_rewards = {i:0 for i in env.possible_agents} #i can report this
                #trace = tensor.zeros((self.context_len,self.qnet_config_dict['ob_space']))
                action=1
                #return2g = 110
                
                

                for i in self.hero_agents_list:
                    self.hero_agents_list[i].model.init_CL_sample_store()





                
                
                if fault_condition:
                    env = env_risk(**(self.env_config  #| {"render_mode" : None,"bad_mov_penalization" : 0.01,"render_":False#False
                                                        # }
                                     )
                                      )#game.env(render_mode=None)

                curren_epi = episode + (iteration-1)*self.num_episodes
                env.reset(curren_epi) #for riplication
                
                fault_condition = False
                step_count = 0
                
                self.reset_moves_hero_agents()
                is_draw = 0
                
                #draw_territory_count = 0
                #is_third = 0

                for agent in env.agent_iter():
                    e_t = env.terminations
                    if sum(e_t.values()) <(self.total_agents-1):
                        observation, reward, termination, truncation, info = env.last()
        
                        observation['observation'] =  self.obs_converter(
                                                        torch.tensor(
                                                            observation['observation']
                                                        ).to(self.device,dtype=torch.float32),
                                                        num_classes = self.total_agents+1)
                        
                        observation['observation'][:,-1]  =  (observation['observation'][:,-1] - 5.2496)/1.4733



                        
                        
                        episodes[step_count] = curren_epi
                        obs[step_count] = observation['observation'] #torch.Tensor(observation['observation']).to(self.device) #sould i not add it .... meaning this is the last observation after the player dies
                        action_masks[step_count] = torch.Tensor(observation['action_mask']).to(self.device)
                        
                        #curr_agent = agent#int(agent[-1])
                        current_agent[step_count] = curr_agent = agent
                        current_phase[step_count] = phase = env.phase_selection
                        phase_mapping = self.map_agent_phase_hot(phase,num_classes = self.total_phases).float()
                        
                        curr_agent_mapping = self.map_agent_phase_hot(int(curr_agent)-1,
                                                                      num_classes = self.total_agents 
                                                                     ).float()
                        
                        current_troops_count[step_count] = torch.tensor([env.board.agents[i].bucket for i in env.possible_agents],requires_grad =False).to(self.device)
                    

                        #model_in = torch.Tensor(torch.hstack((observation['observation'].reshape(-1),
                        #                                      torch.tensor(observation['action_mask'].reshape(-1)).to(self.device)*(curr_agent == self.hero),
                        #                   phase_mapping,
                        #                   curr_agent_mapping,
                        #                   torch.tensor([env.board.agents[ self.hero#curr_agent
                        #                                                ].bucket ]).to(self.device)))[None,:]#.repeat(3,axis = 0)
                        #                        ).float()


                        

                        for i in self.hero_agents_list:
                            self.hero_agents_list[i].model.current_model_in(observation,curr_agent,
                                                                            phase_mapping,curr_agent_mapping,
                                                                            env_board_agents=env.board.agents)
                            self.hero_agents_list[i].model.update_CL_sample_store(curr_agent_=i,
                                                                                  inp = {'step':step_count,
                                                                                         'act_2_1':[] ,
                                                                                         'act_2_2':[] ,
                                                                                         'curr_reward_list':[]
                                          },before_action=1)
                            
                        
                        #if e_t[curr_agent]:
                            #print('heeee')
                        action_taken = False
                        log_prob_a2 = None  
                        if True:
                            
                            mask = observation["action_mask"]
                            if (self.global_step < self.args.learning_starts) or (
                                np.random.rand() > min(
                                                ((curren_epi)/((self.num_iterations*self.num_episodes)/10))
                                                , 0.95)
                                                #) or (agent != self.the_hero_agent) 
                                                ) or ( agent not in self.hero_agents_list):
        
                                
                                action = env.action_space(agent).sample()
                                
                                #part_0 =np.random.choice(np.where(env.board.calculated_action_mask(agent))[0])
                                part_0 =np.random.choice(np.where(observation['action_mask'])[0])
                                action = torch.tensor([
                                                        [
                                                         [part_0],
                                                         [np.around(action[1],2)]
                                                        ]
                                                        ],requires_grad =False).to(self.device)
                                
                                action = action[:,:,0]
                                action_1 = action[:,0]
                                action_2 = action[:,1]
                                log_prob_a2 = torch.tensor([  1/(sum(mask)+0.0001) ],requires_grad =False).to(self.device)
                                for i in self.hero_agents_list:
                                    
                                    self.hero_agents_list[i].model.update_CL_sample_store(curr_agent_=curr_agent,
                                                                                          inp = {'step':step_count,
                                                                                                 'act_2_1':action_1 ,
                                                                                                 'act_2_2':[] ,
                                                                                                 'curr_reward_list':[]
                                                  },before_action=2)
                                    
                                    self.hero_agents_list[i].model.update_CL_sample_store(curr_agent_=curr_agent,
                                                                                          inp = {'step':step_count,
                                                                                                 'act_2_1':[] ,
                                                                                                 'act_2_2':action_2 ,
                                                                                                 'curr_reward_list':[]
                                                  },before_action=3)

                                


                            

                                                              
                            else:
                                action_taken = True
                                #action_masks[step]

                                #need to update this
                                action_1,_,action_taken,_ = self.hero_agents_list[curr_agent].action_predict(save_R=False,action_masks=action_masks[step_count])
                                #action_1 = action_1[None,:]
                                #so the action_1 should be updated here... and prediction of action_2 would be re-done


                                
                                
                                if action_taken:
                                    current_agent_acting[step_count] = curr_agent

                                for i in self.hero_agents_list:
                                    
                                    self.hero_agents_list[i].model.update_CL_sample_store(curr_agent_=curr_agent,
                                                                                          inp = {'step':step_count,
                                                                                                 'act_2_1':action_1 ,
                                                                                                 #'act_2_1':action_1[0] ,
                                                                                                 'act_2_2':[] ,
                                                                                                 'curr_reward_list':[]
                                                  },before_action=2)

                                _,action_2,_,log_prob_a2 = self.hero_agents_list[curr_agent].action_predict(save_R=False,action_masks=action_masks[step_count],return_log_prob_a2 = True)
                               # action_2 = action_2[None,:]


                                for i in self.hero_agents_list:
                                    
                                    self.hero_agents_list[i].model.update_CL_sample_store(curr_agent_=curr_agent,
                                                                                          inp = {'step':step_count,
                                                                                                 'act_2_1':[] ,
                                                                                                 #'act_2_2':action_2[0] ,
                                                                                                 'act_2_2':action_2 ,
                                                                                                 'curr_reward_list':[]
                                                  },before_action=3)
                                    



                            
                                #print(action,action.requires_grad)
                                #if len(action.shape)<2:
                                    #print(episode,"---2--",action, action.shape)
                                    #a()
                                    #break
                                
                            
                                #action = self.actor(torch.Tensor(model_in).to(self.device))

                            #this is only to predict and save the return to go for all the transformer agents, will optimize later
                            _ =  [ self.hero_agents_list[i].action_predict(save_R=True, action_masks=action_masks[step_count] # it only matters for the correct agent ... we are only saving the Q

                                                                           
                                                                          
                                                                          ) for i in self.hero_agents_list]

                            
                            actions_1[step_count] = action_1
                            actions_2[step_count] = action_2
                            if log_prob_a2 != None:
                                log_probs_actions_2[step_count] = log_prob_a2
                            curr_agent_ = int(curr_agent)
                            
                            
        
                            if not observation['action_mask'][action_1.long()]: 
                                fault_condition =True
                                
                                #self.faulting_player = agent

                                


                                if  curr_agent_ in self.hero_agents_list:
                                    self.hero_agents_list[curr_agent_].bad_move_count+=1
                                    self.hero_agents_list[curr_agent_].bad_move_phase_count[int(current_phase[step_count][0])]+=1  # when is the where_is_it_performing_bad_really
                                    #print('here',agent, action, observation['action_mask'])
                            
        
                            if  curr_agent_ in self.hero_agents_list:
                                self.hero_agents_list[curr_agent_].move_count[int(current_phase[step_count][0])]+=1  
                            #if self.the_hero_agent == curr_agent:
                                #move_count[int(current_phase[step][0])]+=1        
        
                        





                            
                        #print('here',agent, action)
                        if action_1 != None :
                            act_2_1 = action_1[0]
                            act_2_2 = action_2[0]
                            
                            #act_2 = action.detach().cpu().numpy()[0]#list([action.detach().cpu().numpy()[0][0], max(action.detach().cpu().numpy()[0][1],0.1) ])
                            #act_2 = torch.tensor([act_2[0], max(act_2[1],0.001) ]).to(device = self.device)
                            #print(max(act_2_2.clone().detach().cpu().item(),0.001))
                            env.step([act_2_1.clone().detach().cpu().item(), max(act_2_2.clone().detach().cpu().item(),0.001) ])  
                            try:
                                _ =1
                                #env.step([act_2_1.clone().detach().cpu().item(), max(act_2_2.clone().detach().cpu().item(),0.001) ])    
                            except Exception as e:
                                print("action_taken",action_taken)
                                print(e)
                                
                                print([act_2_1.clone().detach().cpu(), max(act_2_2.clone().detach().cpu(),0.001) ])
                                
                            
                        #env.step(act_2 if action != None else None)        
        
        
                        if True:
        
                            
                            curr_reward_list =  env.curr_rewards
                            
                            if (step_count == (self.episode_time_lim-1))   or (self.global_step == (self.num_steps-1)): # draw reward
                                is_draw=1
                                curr_reward_list = {i:-100 for i in env.possible_agents }

                            

                            #if self.hero == curr_agent_:
                            #    DT_input['action'][-1]  =act_2
                            #DT_input['return_to_go'][-1] -=  curr_reward_list[self.hero]
                            #returntogo[step] = DT_input['return_to_go'][-1]

                            for i in self.hero_agents_list:
                                self.hero_agents_list[i].model.update_CL_sample_store(curr_agent_=curr_agent,
                                              inp = {'step':step_count,
                                                     'act_2_1':None ,
                                                     'act_2_2':None ,
                                                     'curr_reward_list':curr_reward_list[i]
                                              },before_action=4)





                            
                            rewards_2[step_count] = torch.tensor([curr_reward_list[i] for i in env.possible_agents]).to(self.device,dtype=torch.float32)
                            if step >1:
                                dones_2[step_count] = torch.tensor([ int(env.terminations[i]) - dones_2[step_count-1,i-1]  for i in env.possible_agents]).to(self.device)
                            else:
                                dones_2[step_count] = torch.tensor([env.terminations[i] for i in env.possible_agents]).to(self.device)


                        #list_curr_reward_list = np.array(list(curr_reward_list.values()))
                        
                        #if sum(curr_reward_list.values()) == -300:
                            #print('here')
                            #is_draw=1
        
                        
                        for age_i in env.possible_agents:
                            
                            total_rewards[age_i]+=curr_reward_list[age_i] #env.curr_rewards[age_i] if (step_count != episode_time_lim) else -100
                                    
                        
                        step +=1
                        self.global_step+=1
        
                    else:
                        self.training_performance_return.append(total_rewards[1])
                        print('done:',env.terminations,#env.terminations.values(),
                              ",total_reward:",total_rewards, ',iteration:',iteration,",episode:", episode )
                        print(env.board.territories)
                        break    
                
        
        
        
                    step_count+=1
                    
                    if (self.global_step == self.num_steps) :# or (fault_condition and (fa ulting_player != agent) and (len(env.agents)==0)):
                        
                        print('global_break_1')
                        self.training_performance_return.append(total_rewards[1])
                        print('done:',env.terminations,#env.terminations.values(),
                              ",total_reward:",total_rewards, ',iteration:',iteration,",episode:", episode )
                        print(env.board.territories)

                        #have to get out of the outer loop
                        break
                    elif (step_count == self.episode_time_lim):
                        print('episode_break_1')
                        self.training_performance_return.append(total_rewards[1])
                        print('done:',env.terminations,#env.terminations.values(),
                              ",total_reward:",total_rewards, ',iteration:',iteration,",episode:", episode )
                        print(env.board.territories)
                        break

                #self.obs=obs
                
                #print_here
                print(episode,step_count,iteration)



                
                #print(rewards[step-2])
                if self.global_step == self.num_steps:
                    print('global_break_2')
                    break 

                for i in self.hero_agents_list:
                    self.hero_agents_list[i].position =self.total_agents
                    
                    
                #[ position = 3 for i in ] 
                for k_,(i_,j_) in enumerate(sorted([(j_,i_) for i_,j_ in total_rewards.items()],reverse=True) 
                      ):
                    if int(j_) in self.hero_agents_list:
                        self.hero_agents_list[int(j_)].position = k_+1
                        print(j_,self.hero_agents_list[int(j_)].position)
                        
                        
                    #if j_==self.the_hero_agent:
                    #    position = k_+1

                cur_epi_list = (episodes == curren_epi)
                        
                if self.args.TB_log:
                    self.write_exploring(is_draw,#position,
                            curren_epi,step,
                            step_count,
                            total_rewards,#bad_move_count
                            #,bad_move_phase_count,
                            #move_count,
                            observation,
                            env,
                            cur_epi_list,
                            current_agent_acting)

                #paths = []

                for i in self.hero_agents_list:
                    self.hero_agents_list[i].model.update_train_data(
                         step_count,
                         obs,
                            self.ob_space_shape,
                            rewards_2[:,i-1],
                            dones_2[:,i-1],
                            actions_1[cur_epi_list],actions_2,log_probs_actions_2,
                            action_masks,
                            current_agent,
                            current_agent_acting,
                            current_phase,
                            current_troops_count[:,i-1],
                            map_agent_phase_vector = self.map_agent_phase_vector,
                            
                         )



        #this  ---------------
        #avg_episode_length = torch.mean(torch.tensor(
        #                    [(episodes[:step] == i_epi).sum() for i_epi in episodes[:step].unique()]).float())#np.mean([(episodes[:step] == i_epi).sum() for i_epi in episodes[:step].unique()])
        #if self.args.TB_log:
        #    self.writer.add_scalar("charts/avg_episodic_length", avg_episode_length, self.global_step)



        #return paths
        #return rb

    def update_hero_rb(self,b_obs_a_all,t_next,t_range,current_agent,actions,
                                rewards,dones,infos,current_troops_count,
                                step):



            for agent in self.hero_agents_list:
                #the_hero_agent = agent
                
                
                hero_steps = [current_agent == agent][0][:,0][:step]
                next_indecies = (t_next[:step,agent-1].to(dtype=torch.int) + t_range +1)[hero_steps].long()
                selected_t_next = t_next[:,agent-1,None]#t_next[:step,0,None][hero_steps]

                b_obs_a = torch.concat((b_obs_a_all,
                                        current_troops_count[:,agent-1,None],
                                        selected_t_next),axis =1)
                
                for i in zip(b_obs_a[:step][hero_steps].cpu().to(dtype=torch.float), 
                             b_obs_a[next_indecies].cpu().to(dtype=torch.float), 
                            actions[:step][hero_steps].cpu().to(dtype=torch.float32),
                             rewards[:step][hero_steps][:,agent-1,None].cpu(), 
                           dones[:step][hero_steps][:,agent-1,None].cpu(), infos):
                            self.hero_agents_list[agent].rb.add(*i)
        



        
        

    def write_exploring(self,is_draw,#position,
                        curren_epi,step,
                        step_count,
                        total_rewards,#bad_move_count
                        #,bad_move_phase_count,
                        #move_count,
                        observation,
                        env,
                        cur_epi_list,
                        current_agent_acting):

        if is_draw:
            self.draw_count +=1

            for i in self.hero_agents_list:

                self.writer.add_scalar(f"draw_charts_agent_{i}/position_draw",self.hero_agents_list[i].position
                                                                                            ,self.draw_count) #draw_count is the number of draw episodes




                
                self.hero_agents_list[i].draw_territory_count = int(observation['observation'][:,i].sum()) #this is the total number of states
                
                self.hero_agents_list[i].count_draw_dict[
                                                        self.hero_agents_list[i].position
                                                        ] +=1
                self.writer.add_scalar(f"draw_charts_agent_{i}/draw_territory_count",
                                                                               self.hero_agents_list[i].draw_territory_count,
                                                                               self.draw_count)#self.global_step)
                
                for j in self.hero_agents_list[i].count_draw_dict:
                    self.writer.add_scalar(f"draw_charts_agent_{i}/{j}_position_prop_draw",int(
                                                                                            self.hero_agents_list[i].position==j
                                                                                            ),self.draw_count)

                    if j not in [1,2]:
                        self.writer.add_scalar(f"win_charts_agent_{i}/{j}_position_all_prop",int(
                                                                                            self.hero_agents_list[i].position==j
                                                                                            ),(curren_epi+1))

                        self.writer.add_scalar(f"draw_charts_agent_{i}/{j}_place_in_draw",
                                                                   self.hero_agents_list[i].count_draw_dict[j],
                                                                   self.draw_count)

                        self.writer.add_scalar(f"draw_charts_agent_{i}/{j}_place_in_draw_ratio",
                                                                   self.hero_agents_list[i].count_draw_dict[j]/self.draw_count,
                                                                   self.draw_count)
                        
                    
            self.writer.add_scalar("draw_charts/draw_count",self.draw_count,(curren_epi +1))#self.global_step)
            self.writer.add_scalar("draw_charts/draw",1,(curren_epi+1))
            self.writer.add_scalar("draw_charts/draw_to_total_count",self.draw_count/(curren_epi +1+0.000001),(curren_epi +1))#self.global_step)
            
        else:
            
            non_draw_count =(curren_epi-self.draw_count+1+0.000001)
            for i in self.hero_agents_list:

                self.writer.add_scalar(f"win_charts_agent_{i}/position_win",self.hero_agents_list[i].position
                                                                                            ,(curren_epi+1))



                
                self.hero_agents_list[i].count_dict[self.hero_agents_list[i].position
                                               ] +=1

                
                for j in self.hero_agents_list[i].count_dict:
                    self.writer.add_scalar(f"win_charts_agent_{i}/{j}_position_prop",int(
                                                            self.hero_agents_list[i].position==j
                                                            ),(curren_epi+1))

                    self.writer.add_scalar(f"win_charts_agent_{i}/{j}_position",self.hero_agents_list[i].count_dict[j],
                                           (curren_epi+1))

                    self.writer.add_scalar(f"win_charts_agent_{i}/{j}_position_to_total_terminated",self.hero_agents_list[i].count_dict[j]/non_draw_count,(curren_epi+1))

                    if j not in [1,2]:
                        self.writer.add_scalar(f"win_charts_agent_{i}/{j}_position_all_prop",int(
                                                                                            self.hero_agents_list[i].position==j
                                                                                            ),(curren_epi+1))


            self.writer.add_scalar("draw_charts/draw",0,(curren_epi+1))
            
        for i in self.hero_agents_list:

            self.writer.add_scalar(f"moves/agent_{i}/model_move_count_per_episode",   sum(current_agent_acting[:step_count] ==i)  ,  (curren_epi +1))

            self.writer.add_scalar(f"moves/agent_{i}/model_2_total_move_count_per_episode",   sum(current_agent_acting[:step_count] ==i)/sum( self.hero_agents_list[i].move_count.values()),  (curren_epi +1))

            self.writer.add_scalar(f"win_charts_agent_{i}/position_all",self.hero_agents_list[i].position
                                                                                            ,(curren_epi+1))

            
            for j in self.hero_agents_list[i].count_dict:
                self.writer.add_scalar(f"win_charts_agent_{i}/{j}_position_to_total",(
                                                            self.hero_agents_list[i].count_dict[j]+
                                                            self.hero_agents_list[i].count_draw_dict[j]
                                                        
                                                            )/(curren_epi +1+0.00001 ),(curren_epi+1))#global_step)

            
            self.writer.add_scalar(f"moves/agent_{i}/bad_move_count_per_episode",
                                               self.hero_agents_list[i].bad_move_count,            (curren_epi +1))#self.global_step)
                
            self.writer.add_scalar(f"moves/agent_{i}/bad_move_count_position_per_episode",
                                               self.hero_agents_list[i].bad_move_phase_count[0],            (curren_epi +1))#self.global_step)
            self.writer.add_scalar(f"moves/agent_{i}/bad_move_count_attack_per_episode",
                                               self.hero_agents_list[i].bad_move_phase_count[1],            (curren_epi +1))#self.global_step)
            self.writer.add_scalar(f"moves/agent_{i}/bad_move_count_fortify_per_episode",
                                               self.hero_agents_list[i].bad_move_phase_count[2],            (curren_epi +1))#self.global_step)
            
            self.writer.add_scalar(f"moves/agent_{i}/total_moves",sum(
                                                    self.hero_agents_list[i].move_count.values()),            (curren_epi +1))#self.global_step)
            self.writer.add_scalar(f"moves/agent_{i}/bad_move_to_step_count_per_episode",
                                               self.hero_agents_list[i].bad_move_count/(sum(
                                                   self.hero_agents_list[i].move_count.values())+1),            (curren_epi +1))#self.global_step)
            self.writer.add_scalar(f"moves/agent_{i}/bad_move_to_step_position_per_episode",
                                               self.hero_agents_list[i].bad_move_phase_count[0]/( 
                                                   self.hero_agents_list[i].move_count[0]+1),            (curren_epi +1))#self.global_step)
            self.writer.add_scalar(f"moves/agent_{i}/bad_move_to_step_attack_per_episode",
                                               self.hero_agents_list[i].bad_move_phase_count[1]/( 
                                                   self.hero_agents_list[i].move_count[1]+1),            (curren_epi +1))#self.global_step)
            self.writer.add_scalar(f"moves/agent_{i}/bad_move_to_step_fortify_per_episode",
                                               self.hero_agents_list[i].bad_move_phase_count[2]/( 
                                                   self.hero_agents_list[i].move_count[2]+1),            (curren_epi +1))#self.global_step)

        
        self.writer.add_scalar("charts/epsilon",(curren_epi/((self.num_iterations*self.num_episodes)/10)),(curren_epi +1))#self.global_step)
        self.writer.add_scalar("charts/avg_per_epi_total_reward", np.mean(list(total_rewards.values())), (curren_epi +1))#self.global_step)

        

        #values_total = {i:0 for i in self.env.possible_agents}
        

        
        self.writer.add_scalar("charts/episodic_length", cur_epi_list[:step].sum(), (curren_epi +1))#self.global_step)
        
        for i in env.possible_agents:
            #cur_index = torch.where((current_agent[:,0] == i) &( cur_epi_list ))[0]

            #values_total[i] = values[cur_index].mean()
            #writer.add_scalar("charts/mean_value_per_epi_agent_"+str(i), values_total[i], global_step)
            
            self.writer.add_scalar("charts/total_reward_per_epi_agent_"+str(i), total_rewards[i], (curren_epi +1))#self.global_step)

    
    


        
        




In [13]:
25e3

25000.0

In [14]:
def discount_cumsum(x, gamma,Torch = True):
    if Torch:
        disc_cumsum = torch.zeros_like(x,dtype = torch.float32)
    else:
        disc_cumsum = np.zeros_like(x,dtype = np.float32)
    
    disc_cumsum[-1] = x[-1]
    #print(disc_cumsum[-1])
    for t in reversed(range(x.shape[0]-1)):
        disc_cumsum[t] = x[t] + gamma * disc_cumsum[t+1]
        #print(x[t],disc_cumsum[t])
    return disc_cumsum

In [15]:

exp_3 = dict(
exp_name = 'exp3_ddqn_lr_bs_1',
learning_rate = 0.001,
batch_size = 512,
gamma = 0.99,
num_steps=120000,
num_iterations = 500,
episode_time_lim = 10000,
hero_agent_count = 2,
entropy=False,
return_prob=False,
small = False,
rtg_scale=1,
shuffle=True,
pin_memory=True,
drop_last=True,
num_episodes = 10,
    
env_config = dict(render_mode = None,#'rgb_array', 
                    default_attack_all  = True,
                    agent_count  = 4
                    ,use_placement_perc=True,
                    render_=False,
                    bad_mov_penalization = 0.01
                 
                     #| {"render_mode" : None,"bad_mov_penalization" : 0.01,"render_":False#False
                     #                                   }
                 
                 )

)

exp_4 = dict(
exp_name = 'exp4_ddqn_2_agents_1_hero',
learning_rate = 0.001,
batch_size = 512,
gamma = 0.99,
num_steps=120000,
num_iterations = 500,
episode_time_lim = 10000,
hero_agent_count = 1,
entropy=False,
return_prob=False,
small = False,
rtg_scale=1,
shuffle=True,
pin_memory=True,
drop_last=True,
num_episodes = 10,
env_config = dict(render_mode = None,#'rgb_array', 
                    default_attack_all  = True,
                    agent_count  = 2
                    ,use_placement_perc=True,
                    render_=False,
                    bad_mov_penalization = 0.01
                 )

)


exp_5 = dict(
exp_name = 'exp5_ddqn_2_agents_1_hero',
learning_rate = 0.001,
batch_size = 512,
gamma = 0.99,
num_steps=120000,
num_iterations = 500,
episode_time_lim = 10000,
hero_agent_count = 1,
entropy=False,
return_prob=False,
small = False,
rtg_scale=1,
shuffle=True,
pin_memory=True,
drop_last=True,
num_episodes = 10,
env_config = dict(render_mode = None,#'rgb_array', 
                    default_attack_all  = True,
                    agent_count  = 2
                    ,use_placement_perc=True,
                    render_=False,
                    bad_mov_penalization = 0.1
                 )

)


exp_6 = dict(
exp_name = 'exp6_ddqn_2_agents_1_hero',
learning_rate = 0.0003,
batch_size = 512,
gamma = 0.99,
num_steps=120000,
num_iterations = 500,
episode_time_lim = 10000,
hero_agent_count = 1,
entropy=False,
return_prob=False,
small = False,
rtg_scale=1,
shuffle=True,
pin_memory=True,
drop_last=True,
num_episodes = 10,
env_config = dict(render_mode = None,#'rgb_array', 
                    default_attack_all  = True,
                    agent_count  = 2
                    ,use_placement_perc=True,
                    render_=False,
                    bad_mov_penalization = 0.1
                 )

)


exp_7 = dict(
exp_name = 'exp7_ddqn_2_agents_1_hero',
learning_rate = 0.0003,
batch_size = 512,
gamma = 0.99,
num_steps=120000,
num_iterations = 500,
episode_time_lim = 10000,
hero_agent_count = 1,
entropy=True,
return_prob=2,
actor_wt = 0.5,
CE_wt = 0.01,
small = False,
rtg_scale=1,
shuffle=True,
pin_memory=True,
drop_last=True,
num_episodes = 10,
env_config = dict(render_mode = None,#'rgb_array', 
                    default_attack_all  = True,
                    agent_count  = 2
                    ,use_placement_perc=True,
                    render_=False,
                    bad_mov_penalization = 0.1
                 )

)

exp_8 = dict(
exp_name = 'exp8_ddqn_2_agents_1_hero_norm',
learning_rate = 0.0003,
batch_size = 512,
gamma = 0.99,
num_steps=120000,
num_iterations = 500,
episode_time_lim = 10000,
hero_agent_count = 1,
entropy=True,
return_prob=2,
actor_wt = 0.5,
CE_wt = 0.01,
small = False,
rtg_scale=1,
shuffle=True,
pin_memory=True,
drop_last=True,
num_episodes = 10,
env_config = dict(render_mode = None,#'rgb_array', 
                    default_attack_all  = True,
                    agent_count  = 2
                    ,use_placement_perc=True,
                    render_=False,
                    bad_mov_penalization = 0.1
                 )

)

exp_9 = dict(
exp_name = 'exp9_ddqn_4_agents_2_hero_norm',
learning_rate = 0.0003,
batch_size = 512,
gamma = 0.99,
num_steps=120000,
num_iterations = 500,
episode_time_lim = 10000,
hero_agent_count = 2,
entropy=True,
return_prob=2,
actor_wt = 0.5,
CE_wt = 0.01,
small = False,
rtg_scale=1,
shuffle=True,
pin_memory=True,
drop_last=True,
num_episodes = 10,
env_config = dict(render_mode = None,#'rgb_array', 
                    default_attack_all  = True,
                    agent_count  = 4
                    ,use_placement_perc=True,
                    render_=False,
                    bad_mov_penalization = 0.1
                 )

)




exp_10 = dict(
exp_name = 'exp10_ddqn_4_agents_2_hero_norm_small',
learning_rate = 0.0003,
batch_size = 512,
gamma = 0.99,
num_steps=120000,
num_iterations = 500,
episode_time_lim = 10000,
hero_agent_count = 2,
entropy=True,
return_prob=2,
actor_wt = 0.5,
CE_wt = 0.01,
small = True,
rtg_scale=1,
shuffle=True,
pin_memory=True,
drop_last=True,
num_episodes = 10,
env_config = dict(render_mode = None,#'rgb_array', 
                    default_attack_all  = True,
                    agent_count  = 4
                    ,use_placement_perc=True,
                    render_=False,
                    bad_mov_penalization = 0.01
                 )

)


exp_11 = dict(
exp_name = 'exp11_ddqn_4_agents_2_hero_norm_small',
learning_rate = 0.0003,
batch_size = 512,
gamma = 0.99,
num_steps=120000,
num_iterations = 500,
episode_time_lim = 10000,
hero_agent_count = 2,
entropy=True,
return_prob=2,
actor_wt = 0.5,
CE_wt = 0.01,
small = True,

context_len = 64,
rtg_scale=1,
shuffle=True,
pin_memory=True,
drop_last=True,
num_episodes = 5,
env_config = dict(render_mode = None,#'rgb_array', 
                    default_attack_all  = True,
                    agent_count  = 4
                    ,use_placement_perc=True,
                    render_=False,
                    bad_mov_penalization = 0.01
                 )

)



exp_12 = dict(
exp_name = 'exp12_ddqn_4_agents_2_hero_norm_small',
learning_rate = 5e-4,#0.003,#0.0003,
batch_size = 2,
gamma = 0.99,
num_steps=1200000,
num_iterations = 500,
episode_time_lim = 3000,#10000,
hero_agent_count = 1,
model_name={1:"transformer_model"}#,2:'transformer_model'}
,entropy=True,
return_prob=2,
actor_wt = 0.5,
CE_wt = 0.01,
small = True,
num_episodes = 4,
context_len = 256,
rtg_scale=1,
shuffle=True,
pin_memory=False,#False,
drop_last=True,
TB_log=True,
learning_starts =100,
update_epochs = 1,
device = torch.device("cuda" if torch.cuda.is_available() else "cpu"),

pin_memory_device= ("cuda" if torch.cuda.is_available() else "cpu"),

    
env_config = dict(render_mode = None,#'rgb_array', 
                    default_attack_all  = True,
                    agent_count  = 3#4
                    ,use_placement_perc=True,
                    render_=False,
                    bad_mov_penalization = 0.01
                 )
,model_config = dict(
                    n_blocks      =   3,
                    embed_dim     =   64,#128 ,
                    context_len   =   256,#256  ,
                    n_heads       =   1,
                    dropout_p     =   0.1,
                    wt_decay      =   0.0001,
                    warmup_steps  =   100   ,
                    tau           =   0.95,
                    chunk_size    =   64,#8,#32,#64
                    chunk_overlap =   1,
                    rb_len        =   20,
                
                    warmup_epoch  =   5,  
                    total_epoch   =   20,
                    initial_lr    =   5e-4,
                    final_lr      =   1e-6,

                    beta           = 0.5,#args.model_config['beta']         #0.2 #Q_mse
                    alpha          = 1,#args.model_config['alpha']          #0.1  #actionloss
                    entropy_coeff  = 0.2,#args.model_config['entropy_coeff']         #0.1#0.5   #entropy loss in action
                    val_loss_coeff = 0.5#args.model_config['val_loss_coeff']        #0.5      #Q loss
            



    
                    )

)


## training

In [16]:
#T = Trainer(Args,param_dict = exp_3)
T = Trainer(Args,param_dict = exp_12)

tensor([[0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0]], dtype=torch.int8)
tensor([[0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0]], device='cuda:0', dtype=torch.int8)


C:\Users\31721\AppData\Local\Temp\5\ipykernel_56224\2221868122.py:84: UserWarning: expandable_segments not supported on this platform (Triggered internally at ..\c10/cuda/CUDAAllocatorConfig.h:30.)
  print(torch.tensor(self.env.last()[0]['observation']


17
<class 'type'>


In [ ]:
T.run_training_loop()

done: {1: True, 2: True, 3: True} ,total_reward: {1: -200.26999999999998, 2: -99.08, 3: 109.88} ,iteration: 305 ,episode: 0
[[3 3.0]
 [3 4.0]
 [3 1.0]
 [0 0.0]
 [3 3.0]
 [3 1.0]
 [3 12.0]
 [3 1.0]
 [3 8.0]
 [3 1.0]]
0 817 305
1 3
done: {1: True, 2: True, 3: True} ,total_reward: {1: -98.2, 2: 107.99, 3: -200.0} ,iteration: 305 ,episode: 1
[[0 0.0]
 [2 1.0]
 [2 4.0]
 [0 0]
 [2 3.0]
 [2 2.0]
 [2 6.0]
 [2 1.0]
 [2 2.0]
 [2 4.0]]
1 258 305
1 2
done: {1: True, 2: True, 3: True} ,total_reward: {1: -100.73999999999998, 2: -200.07999999999998, 3: 109.78} ,iteration: 305 ,episode: 2
[[0 0.0]
 [3 1.0]
 [3 2.0]
 [3 2.0]
 [3 9.0]
 [3 3.0]
 [3 4.0]
 [3 1.0]
 [3 2.0]
 [3 5.0]]
2 2264 305
1 2
episode_break_1
done: {1: False, 2: True, 3: False} ,total_reward: {1: -95.93999999999998, 2: -200.09, 3: -95.22} ,iteration: 305 ,episode: 3
[[1 5.0]
 [1 2.0]
 [3 21.0]
 [1 85.0]
 [3 1.0]
 [3 7.0]
 [3 2.0]
 [0 0.0]
 [1 24.0]
 [1 1.0]]
3 3000 305
1 2
0
divi 36 chunk_size 64 batch_shape torch.Size([1, 2239, 256])


In [24]:
T.env.board.phase

0

In [ ]:
print(logp_pi_a_2_dir[:, -1,0])
print(log_probs_actions_2[:, -1])

In [23]:
for name, param in T.hero_agents_list[1].model.model.named_parameters():
    if not torch.isfinite(param).all():#torch.isinf(param).any():
      print(f"Infinite value found in weight: {name}")
    if param.grad is not None and torch.isinf(param.grad).any():
      print(f"Infinite value found in gradient: {name}")



- number of steps per episode
- number of bad moves
- total reward
- kind of play
- position
- 

In [ ]:
T.hero_agents_list[1].model.model_in[0]

In [ ]:
T.env.last()[0]['observation']


T.obs_converter(torch.tensor(T.env.last()[0]['observation']#observation['observation']
                                                        ).to(T.device,dtype=torch.float32),
                                                        num_classes = T.total_agents+1)[:,-1]

In [ ]:
for name, param in T.hero_agents_list[1].model.model.named_parameters():
    if torch.isnan(param.grad).any():
        print("nan gradient found",name)
        raise SystemExit

In [ ]:
# normalize states
        #for traj in self.trajectories:
        #    traj['observations'] = (traj['observations'].to(dtype=torch.float32) - self.state_mean) / self.state_std

In [ ]:
self.state_mean) / self.state_std


current_troops_count_mean tensor(5.2496), current_troops_count_std = tensor(1.4733)

(x-mean)/std

In [ ]:
for traj in self.trajectories:

In [ ]:
states = torch.cat((states, action_masks * hero_steps,
                      current_phase, current_agent,
                      current_troops_count[:, :, None]), axis=2) 

In [19]:
dat_ = [batch for batch in T.hero_agents_list[1].model.traj_data_loader][0]

In [23]:
(
            timesteps,
            states,
            actions_1,
            actions_2,log_probs_actions_2,
            returntogo,
            returns_to_go_cal,
            returntogo_pred,
            reward,
            traj_mask,
            action_masks,
            current_agent_acting,
            current_agent_simple,
            current_agent,
            current_phase,
            current_troops_count,
        ) = dat_

In [23]:
log_probs_actions_2[0,:,-1].sum()

tensor(0.5293, device='cuda:0')

In [21]:
actions_2.shape

torch.Size([1, 2580, 256])

In [23]:
with torch.no_grad():
    for batch in T.hero_agents_list[1].model.traj_data_loader:
        (
            timesteps,
            states,
            actions_1,
            actions_2, lp,
            returntogo,
            returns_to_go_cal,
            returntogo_pred,
            reward,
            traj_mask,
            action_masks,
            current_agent_acting,
            current_agent_simple,
            current_agent,
            current_phase,
            current_troops_count,
        ) = batch
        print(actions_2.shape)
        if (actions_2[0,-1,:] >1).any():
            
            print(actions_2[0,-1,:])

torch.Size([1, 7825, 256])
torch.Size([1, 797, 256])
torch.Size([1, 1753, 256])
torch.Size([1, 1021, 256])
torch.Size([1, 10000, 256])
torch.Size([1, 10000, 256])
torch.Size([1, 10000, 256])
torch.Size([1, 2464, 256])
torch.Size([1, 2413, 256])
torch.Size([1, 10000, 256])
torch.Size([1, 7611, 256])
torch.Size([1, 10000, 256])
torch.Size([1, 4520, 256])
torch.Size([1, 619, 256])
torch.Size([1, 7846, 256])
torch.Size([1, 3012, 256])
torch.Size([1, 10000, 256])
torch.Size([1, 10000, 256])
torch.Size([1, 8041, 256])
torch.Size([1, 10000, 256])


In [ ]:
da

In [27]:
dat_ =[batch for batch in T.hero_agents_list[1].model.traj_data_loader][3]

In [20]:
(
            timesteps,
            states,
            actions_1,
            actions_2,
            returntogo,
            returns_to_go_cal,
            returntogo_pred,
            reward,
            traj_mask,
            action_masks,
            current_agent_acting,
            current_agent_simple,
            current_agent,
            current_phase,
            current_troops_count,
        ) =dat_

In [22]:
returns_to_go_cal[0,:,-1]

tensor([4.5261, 4.5718, 4.6180, 4.6646, 4.7118, 4.7594, 4.8074, 4.8560, 4.9050,
        4.9546, 5.0046, 5.0552, 5.1062, 4.1477, 4.1896, 4.2319, 4.2747, 4.3179,
        4.3615, 4.4055, 4.4500, 4.4950, 4.5404, 4.5862, 4.6326, 4.6794, 4.7266,
        4.7744, 4.8226, 4.8713, 4.9205, 4.9702, 5.0204, 5.0711, 5.1224, 5.1741,
        5.2264, 5.2792, 5.3325, 4.3762, 4.4205, 4.4651, 4.5102, 4.5659, 3.6019,
        3.6383, 3.6750, 3.7121, 3.7496, 3.7875, 3.8258, 3.8644, 3.9034, 3.9429,
        3.9827, 4.0229, 4.0636, 4.1046, 4.1461, 4.1880, 4.2303, 4.2730, 4.3161,
        4.3597, 3.3937, 2.4179, 2.4423, 2.4670, 2.4919, 2.5170, 2.5425, 2.5681,
        2.5941, 2.6203, 2.6468, 2.6735, 2.7005, 2.7278, 2.7553, 2.7832, 2.8113,
        2.8397, 2.8684, 2.8973, 2.9266, 2.9562, 2.9860, 3.0162, 3.0466, 3.0774,
        3.1085, 3.1399, 3.1716, 3.2037, 3.2360, 3.2687, 3.3017, 3.3351, 3.3688,
        3.4028, 3.4372, 3.4719, 3.5069, 3.5424, 3.5781, 3.6143, 3.6508, 2.6776,
        1.6945, 1.7116, 1.7289, 1.7464, 

In [ ]:
divi 3 chunk_size 256 batch_shape torch.Size([1, 578, 256])
1
divi 7 chunk_size 256 batch_shape torch.Size([1, 1558, 256])
2
divi 8 chunk_size 256 batch_shape torch.Size([1, 1861, 256])
3
divi 7 chunk_size 256 batch_shape torch.Size([1, 1691, 256])
4
divi 4 chunk_size 256 batch_shape torch.Size([1, 959, 256])
5
divi 11 chunk_size 256 batch_shape torch.Size([1, 2583, 256])
6
divi 10 chunk_size 256 batch_shape torch.Size([1, 2483, 256])
7
divi 10 chunk_size 256 batch_shape torch.Size([1, 2447, 256])
8
divi 17 chunk_size 256 batch_shape torch.Size([1, 4308, 256])

In [ ]:

current_troops_count_std,current_troops_count_mean = [], []

with torch.no_grad():
    for batch in T.hero_agents_list[1].model.traj_data_loader:
        (
            timesteps,
            states,
            actions_1,
            actions_2,
            returntogo,
            returns_to_go_cal,
            returntogo_pred,
            reward,
            traj_mask,
            action_masks,
            current_agent_acting,
            current_agent_simple,
            current_agent,
            current_phase,
            current_troops_count,
        ) = dat_

        current_troops_count_std.append(current_troops_count[:torch.where(action_masks[0,:,-1,:].sum(-1) != 0)[0][-1]+1].std())
        current_troops_count_mean.append(current_troops_count[:torch.where(action_masks[0,:,-1,:].sum(-1) != 0)[0][-1]+1].mean())
        

torch.mean(torch.tensor(current_troops_count_std)),torch.mean(torch.tensor(current_troops_count_mean))
        




In [ ]:












(
    timesteps,
    states,
    actions_1,
    actions_2,
    returntogo,
    returns_to_go_cal,
    returntogo_pred,
    reward,
    traj_mask,
    action_masks,
    current_agent_acting,
    current_agent_simple,
    current_agent,
    current_phase,
    current_troops_count,
    ) = dat_


In [ ]:
torch.isfinite(returns_to_go_cal).all()

In [22]:
torch.where(action_masks[0,:,-1,:].sum(-1) != 0)[0][-1]+1

tensor(2580, device='cuda:0')

In [24]:

device = 'cpu'#torch.device("cuda" if torch.cuda.is_available() else "cpu")


timesteps = timesteps[0,:2580].to(device)  # B x T
states = states[0,:2580].to(device)  # B x T x state_dim
actions_1 = actions_1[0,:2580].to(device)  # B x T x act_dim
actions_2 = actions_2[0,:2580].to(device) 
reward = reward[0,:2580].to(device)
returns_to_go_cal =   returns_to_go_cal[0,:2580].to(device).unsqueeze(dim=-1)  # B x T x 1
returntogo = returntogo[0,:2580].to(device)
returntogo_pred = returntogo_pred[0,:2580].to(device)
traj_mask = traj_mask[0,:2580].to(device)  # B x T
action_masks = action_masks[0,:2580].to(device)
current_agent_acting = current_agent_acting[0,:2580].to(device)
current_agent_simple = current_agent_simple[0,:2580].to(device)
current_agent = current_agent[0,:2580].to(device)
current_phase = current_phase[0,:2580].to(device)
current_troops_count = current_troops_count[0,:2580].to(device)

info = dict({})

hero_steps = current_agent_simple == 1#self.hero

states = torch.cat((states, action_masks * hero_steps,
                      current_phase, current_agent,
                      current_troops_count[:, :, None]), axis=2)  # ,torch.ones(len(action_masks))[:,None]*self.hero


In [ ]:
T.hero_agents_list[1].model.device

In [ ]:
(timesteps.device,
states.device,
actions_1.device,
actions_2.device,
returntogo.device,)

In [ ]:
actions_1.shape

In [ ]:
#  action_logit_model_1,action_model_2_dir,_) =   self.model.forward(timesteps=timesteps, states=states,
#                        actions_1=actions_1,actions_2=actions_2,
#                        returns_to_go=returns_to_go, print_=2,return_logit=True)  # print_

#                                                 # ,info = info

# actions_1_ = actions_1.clone().detach()#, requires_grad=False)

# # self.actions_ = actions_
# # self.action_preds_model_ = action_preds_model
# # actions_[:,:,-1] = action_preds_model



# actions_1_[:, -1] = action_logit_model_1[:, -1, :
#         ].argmax(axis=1).clone().detach()

In [27]:
actions_go_pred.shape,actions_2[:10].shape

(torch.Size([10, 256, 1]), torch.Size([10, 256]))

In [29]:
a1_,actions_go_pred, returntogo_pred_,lp = T.hero_agents_list[1].model.model.forward(
                                                        timesteps=timesteps[:10],
                                                        states=states[:10],
                                                        actions_1=actions_1[:10],
                                                        actions_2=actions_2[:10],
                                                        returns_to_go=returntogo[:10],
                                                            print_=2,#return_log_prob_a2= True
                                                                return_og_log_prob_a2 = True
                                                        #,info = info
                                                    )

In [30]:
lp[:,-1,0]

tensor([ 0.2009,  0.5237,  0.4434, -0.3261,  0.5088,  0.0081, -0.9799,  0.4514,
        -2.7107, -0.8660], grad_fn=<SelectBackward0>)

In [ ]:
(a1_[:,-1,:][hero_steps[:,-1,0]]*action_masks[:,-1,:][hero_steps[:,-1,0]]).argmax(axis=1)

In [ ]:
action_masks.shape

In [ ]:
actions_go_pred.shape

In [ ]:
actions[0,:,0]

In [ ]:
current_agent_acting[:,-1,0].unique()

In [ ]:
#

In [ ]:
torch.where(actions_go_pred[:,-1,:32].argmax(dim=1) == actions[:,-1,0])

In [ ]:
actions_go_pred[:,-1,:32].argmax(dim=1)


In [ ]:
T.hero_agents_list[1].model.DT_input.keys()

In [ ]:
T.hero_agents_list[1].model.DT_input['return_to_go']

In [ ]:
[i.requires_grad for i in  [batch for batch in T.hero_agents_list[1].model.traj_data_loader][0] ]

In [ ]:
[batch for batch in T.hero_agents_list[1].model.traj_data_loader][0].trajectories[0]['observations']

In [ ]:
[batch for batch in T.hero_agents_list[1].model.traj_data_loader][0]

In [ ]:
T.hero_agents_list[1].model.actions_[:,-1,0] = T.hero_agents_list[1].model.action_preds_model_[:,0,:32].argmax(axis =1)
T.hero_agents_list[1].model.actions_[:,-1,1] = T.hero_agents_list[1].model.action_preds_model_[:,0,32]



In [ ]:
T.hero_agents_list[1].model.action_preds_model_[:,0,:32].argmax(axis =1)

In [ ]:
T.hero_agents_list[1].model.action_preds_model_[:,0,32].shape

In [ ]:
timesteps,                #torch.Size([1, 2475, 256])    
states,                   #torch.Size([1, 2475, 256, 50]) 
actions,                  #torch.Size([1, 2475, 256, 2])  
returntogo,               #torch.Size([1, 2475, 256, 1])     
returns_to_go_cal,        #torch.Size([1, 2475, 256])            
returntogo_pred,          #torch.Size([1, 2475, 256, 1])          
reward,                   #torch.Size([1, 2475, 256]) 
traj_mask ,               #torch.Size([1, 2475, 256])     
action_masks,             #torch.Size([1, 2475, 256, 32])       
current_agent_simple,     #torch.Size([1, 2475, 256, 1])               
current_agent,            #torch.Size([1, 2475, 256, 3])        
current_phase,            #torch.Size([1, 2475, 256, 2])        
current_troops_count      #torch.Size([1, 2475, 256])              

In [ ]:
(timesteps,                #torch.Size([1, 2475, 256])    
states,                   #torch.Size([1, 2475, 256, 50]) 
actions,                  #torch.Size([1, 2475, 256, 2])  
returntogo,               #torch.Size([1, 2475, 256, 1])     
returns_to_go_cal,        #torch.Size([1, 2475, 256])            
returntogo_pred,          #torch.Size([1, 2475, 256, 1])          
reward,                   #torch.Size([1, 2475, 256]) 
traj_mask ,               #torch.Size([1, 2475, 256])     
action_masks,             #torch.Size([1, 2475, 256, 32])       
current_agent_simple,     #torch.Size([1, 2475, 256, 1])               
current_agent,            #torch.Size([1, 2475, 256, 3])        
current_phase,            #torch.Size([1, 2475, 256, 2])        
current_troops_count      #torch.Size([1, 2475, 256])    
)= [batch for batch in T.hero_agents_list[1].model.traj_data_loader][0]

In [ ]:
RT1 = reward_2[:-1] + gamma*returns_target_2[1:]
Q_TD = torch.nn.functional.smooth_l1_loss(return_preds_2[:-1], RT1)
Q_MSE = torch.nn.functional.smooth_l1_loss(return_preds_2, returns_to_go_2)


reward_2 = reward.squeeze().view(-1)[traj_mask.view(-1,) > 0].to(self.device)
return_preds_2 = return_preds.squeeze().view(-1, #act_dim
                                            )[traj_mask.view(-1,) > 0].to(self.device)
returns_target_2 = returns_target.squeeze().view(-1)[traj_mask.view(-1,) > 0].to(self.device)
returns_to_go_2 = returns_to_go.squeeze().view(-1)[traj_mask.view(-1,) > 0].to(self.device)
returns_to_go_cal_2 = returns_to_go_cal.squeeze().view(-1)[traj_mask.view(-1,) > 0].to(self.device)

In [ ]:
traj_mask.shape

In [ ]:
returntogo_pred.squeeze().view(-1, #act_dim
                                            )[traj_mask.view(-1,) > 0].shape

In [ ]:
returns_to_go_cal.squeeze().view(-1)[traj_mask.view(-1,) > 0].shape

In [ ]:
returns_to_go_cal[:,:,-1].squeeze().view(-1).shape

In [ ]:
timesteps.shape

In [ ]:
hero_steps = (current_agent_simple[0] == 1 )

#states_ = torch.concat(
    
(states[0].shape,
         (action_masks[0]*hero_steps).shape,
         current_phase[0].shape,
         current_agent[0].shape,
         #,torch.ones(len(action_masks))[:,None]*self.hero
         current_troops_count[0][:,:,None].shape)
#        ), axis =2)

In [ ]:
T.hero_agents_list[1].model.model.embed_state(states)

In [ ]:
(timesteps,             #torch.Size([1, 7229, 256])      
 states,                #torch.Size([1, 7229, 256, 50])   
 actions,               #torch.Size([1, 7229, 256, 2])    
 returns_to_go,         #torch.Size([1, 7229, 256, 4])         
 reward,                #torch.Size([1, 7229, 256, 4])  
 traj_mask ,            #torch.Size([1, 7229, 256])      
 action_masks,          #torch.Size([1, 7229, 256, 32])        
 current_agent_simple,  #torch.Size([1, 7229, 256, 1])                
 current_agent,         #torch.Size([1, 7229, 256, 3])         
 current_phase,         #torch.Size([1, 7229, 256, 2])         
 current_troops_count   #torch.Size([1, 7229, 256, 4])                
)= [batch for batch in T.hero_agents_list[1].model.traj_data_loader][0]

In [ ]:
returns_to_go[0].to(T.device).unsqueeze(dim=-1).shape

In [ ]:
returns_to_go.shape

In [ ]:
for k in [batch for batch in T.hero_agents_list[1].model.traj_data_loader][0]:
    print(k.shape)

In [ ]:
self.model(timesteps= self.DT_input['timestep'], states=self.DT_input['state'][None,:],
                          actions=self.DT_input['action'],
                          returns_to_go=self.DT_input['return_to_go']
                         )

In [ ]:
T.hero_agents_list[1].model.DT_input['timestep'].long().shape

In [ ]:
T.hero_agents_list[1].model.model.embed_timestep(
T.hero_agents_list[1].model.DT_input['timestep'].long()
).shape

In [ ]:
(T.hero_agents_list[1].model.DT_input['timestep'].shape,
T.hero_agents_list[1].model.DT_input['state'].shape,
T.hero_agents_list[1].model.DT_input['action'].shape,
T.hero_agents_list[1].model.DT_input['return_to_go'].shape)

In [ ]:
T.hero_agents_list[1].model.model.embed_state(
T.hero_agents_list[1].model.DT_input['state'][None,:].float()
).shape

In [ ]:
T.hero_agents_list[1].model.model.embed_state(
T.hero_agents_list[1].model.DT_input['state'].repeat(3,1,1).float()
).shape

In [ ]:
T.hero_agents_list[1].model.model.embed_action_1(
    T.hero_agents_list[1].model.DT_input['action'][:,:,0].long()
).shape

In [ ]:
T.hero_agents_list[1].model.model.embed_action(
torch.concat((
T.hero_agents_list[1].model.model.embed_action_1(
    T.hero_agents_list[1].model.DT_input['action'][:,:,0].long()
    ),

    T.hero_agents_list[1].model.model.embed_action_2(
     T.hero_agents_list[1].model.DT_input['action'][:,:,None][:,:,:,1])), axis =2)).shape



In [ ]:
T.hero_agents_list[1].model.returntogo_pred[T.hero_agents_list[1].model.DT_input['timestep'][0,-1]]


s,a,R = T.hero_agents_list[1].model.model(



    timesteps= T.hero_agents_list[1].model.DT_input['timestep'], 
        states=T.hero_agents_list[1].model.DT_input['state'],
       actions=T.hero_agents_list[1].model.DT_input['action'],
returns_to_go=T.hero_agents_list[1].model.DT_input['return_to_go']
    
)

In [ ]:
T.hero_agents_list[1].model.act_dim

In [ ]:
a[0,0,:32].argmax()

In [ ]:
=

In [ ]:
R[0,0]#.argmax()

In [ ]:

T.hero_agents_list[1].model.model.embed_action(
torch.concat((
T.hero_agents_list[1].model.model.embed_action_1(
    T.hero_agents_list[1].model.DT_input['action'].repeat(3,1,1)[:,:,0].long()
    ),

    T.hero_agents_list[1].model.model.embed_action_2(
     T.hero_agents_list[1].model.DT_input['action'].repeat(3,1,1)[:,:,None][:,:,:,1])), axis =2)).shape


             

In [ ]:
(timesteps, states, actions, returns_to_go,reward, traj_mask ,action_masks,
 current_agent_simple,current_agent,current_phase,current_troops_count) = [i for i in T.traj_data_loader ][0]

In [ ]:
(timesteps, states, actions, returns_to_go,reward, traj_mask ,action_masks,
 current_agent_simple,current_agent,current_phase,current_troops_count) = (
     timesteps[0], states[0], actions[0], returns_to_go[0],reward[0], traj_mask[0] ,action_masks[0],
 current_agent_simple[0],current_agent[0],current_phase[0],current_troops_count[0])

In [ ]:
hero_steps = (current_agent_simple ==1)

In [ ]:
hero_steps = (current_agent_simple ==1)
torch.concat((states,
                 action_masks*hero_steps,
                 
                 current_phase,
                 current_agent
            
                #,torch.ones(len(action_masks))[:,None]*self.hero
                ,current_troops_count[:,:,1,None]
                ), axis =2).shape

In [ ]:
env_config = dict(render_mode = 'rgb_array', default_attack_all  = True,
                render_ = True,agent_count  = 3,use_placement_perc=True)
env = env_risk(**(env_config | {"render_mode" : None,"verbose":False, "agent_count" :4,"render_":False}))


total_rewards = {i:0 for i in env.possible_agents}

step =0
for i__ in range(1):
    env.reset()#seed=42)
    #print(env_1.infos.keys())
    observation, reward, termination, truncation, info = env.last()
    
    total_rewards = {i:0 for i in env.possible_agents}
    
    #so there is an issue ...reward =2 sometimes ... doubling?

    for agent in env.agent_iter():
        step+=1
        e_t = env.terminations
        if sum(e_t.values()) <3:
            observation, reward, termination, truncation, info = env.last()
            
            action = env.action_space(agent).sample()
            
            #if env_1.phase_selection ==1:
            #    action = [int(action[0]),action[1]]
            part_0 =np.random.choice(np.where(env.board.calculated_action_mask(agent))[0])
            #print(part_0)
            
            #action = [part_0,np.around(action[1],2) #min(action[1],env_1.board.agents[agent].bucket ) 
            #                  if env_1.phase_selection==0 else  action[1]]
    
            action = [part_0,np.around(action[1],2)]
    
    
            for i in env.possible_agents:
                total_rewards[i]+=env.curr_rewards[i]
            
            env.step(action)

            if step%10000 ==0:
                print(step)
                print(env.board.territories)
                
                print('\naction',action,
                      '\ninfo',info,
                      '\ninfos',env.infos,
                      '\naction_valid',observation['action_mask'][action[0]],
                      '\nagent',agent,
                      '\nselected_agent',env.agent_selection,
                      '\ncurr_agent',env.board.current_agent,
                      '\ncurr phase',env.phase_selection,
                      '\nbad_trail count',env.board.bad_trials,
                      '\nmax_bad trails', env.board.max_bad_trials,
                      '\nreward',reward,
                      '\nreward',env.curr_rewards,
                      '\ntotal_reward',total_rewards,
                      '\nrewards',env.rewards,
                      '\nbuckets', [env.board.agents[i].bucket for i in env.agents]
                     )
                print('kill_list',env.kill_list,'term',e_t, 'buckets',[i.bucket for i in env.board.agents])
                #if sum(env.curr_rewards.values()) <0:
                #    input()
        else:
            print(env.board.territories)
            print('done',env.terminations.values(),env.curr_rewards)
            break
step


#need to debug .... so the action is valid... but there is no change
#so either the phase is not changing and the agent is not changing... why??


# transformer module

## masked attention and block

In [ ]:
import gymnasium as gym
import os
#import gym
import numpy as np

import collections
import pickle
import tqdm
from stable_baselines3 import PPO


import sys
import random
import csv
from datetime import datetime

import math

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
"""
this extremely minimal GPT model is based on:
Misha Laskin's tweet: 
https://twitter.com/MishaLaskin/status/1481767788775628801?cxt=HHwWgoCzmYD9pZApAAAA

and its corresponding notebook:
https://colab.research.google.com/drive/1NUBqyboDcGte5qAJKOl8gaJC28V_73Iv?usp=sharing

the above colab has a bug while applying masked_fill which is fixed in the
following code

"""

def layer_init(layer, std=np.sqrt(2), bias_const=0.0):
    torch.nn.init.orthogonal_(layer.weight, std)
    torch.nn.init.constant_(layer.bias, bias_const)
    return layer



class MaskedCausalAttention(nn.Module):
    def __init__(self, h_dim, max_T, n_heads, drop_p):
        super().__init__()

        self.n_heads = n_heads
        self.max_T = max_T

        self.q_net = layer_init(nn.Linear(h_dim, h_dim), std=0.01)
        self.k_net = layer_init(nn.Linear(h_dim, h_dim), std=0.01)
        self.v_net = layer_init(nn.Linear(h_dim, h_dim), std=0.01)

        self.proj_net = layer_init(nn.Linear(h_dim, h_dim), std=0.01)

        self.att_drop = nn.Dropout(drop_p)
        self.proj_drop = nn.Dropout(drop_p)

        ones = torch.ones((max_T, max_T))
        mask = torch.tril(ones).view(1, 1, max_T, max_T)

        # register buffer makes sure mask does not get updated
        # during backpropagation
        self.register_buffer('mask',mask)

    def forward(self, x):
        B, T, C = x.shape # batch size, seq length, h_dim * n_heads

        N, D = self.n_heads, C // self.n_heads # N = num heads, D = attention dim

        # rearrange q, k, v as (B, N, T, D)
        q = self.q_net(x).view(B, T, N, D).transpose(1,2)
        k = self.k_net(x).view(B, T, N, D).transpose(1,2)
        v = self.v_net(x).view(B, T, N, D).transpose(1,2)

        # weights (B, N, T, T)
        weights = q @ k.transpose(2,3) / math.sqrt(D)
        # causal mask applied to weights
        weights = weights.masked_fill(self.mask[...,:T,:T] == 0, float('-inf'))
        # normalize weights, all -inf -> 0 after softmax
        normalized_weights = F.softmax(weights, dim=-1)

        # attention (B, N, T, D)
        attention = self.att_drop(normalized_weights @ v)

        # gather heads and project (B, N, T, D) -> (B, T, N*D)
        attention = attention.transpose(1, 2).contiguous().view(B,T,N*D)

        out = self.proj_drop(self.proj_net(attention))
        return out


class Block(nn.Module):
    def __init__(self, h_dim, max_T, n_heads, drop_p):
        super().__init__()
        self.attention = MaskedCausalAttention(h_dim, max_T, n_heads, drop_p)
        self.mlp = nn.Sequential(
                layer_init(nn.Linear(h_dim, 4*h_dim), std=0.01),
                nn.GELU(),
                layer_init(nn.Linear(4*h_dim, h_dim), std=0.01),
                nn.Dropout(drop_p),
            )
        self.ln1 = nn.LayerNorm(h_dim)
        self.ln2 = nn.LayerNorm(h_dim)

    def forward(self, x):
        # Attention -> LayerNorm -> MLP -> LayerNorm
        x = x + self.attention(x) # residual
        x = self.ln1(x)
        x = x + self.mlp(x) # residual
        x = self.ln2(x)
        return x





## DT class

In [ ]:
class DecisionTransformer(nn.Module):
    def __init__(self, state_dim, act_dim, n_blocks, h_dim, context_len, 
                 n_heads, drop_p, max_timestep=4096):
        super().__init__()

        self.state_dim = state_dim
        self.act_dim = act_dim
        self.h_dim = h_dim

        ### transformer blocks
        input_seq_len = 3 * context_len
        blocks = [Block(h_dim, input_seq_len, n_heads, drop_p) for _ in range(n_blocks)]
        self.transformer = nn.Sequential(*blocks)

        ### projection heads (project to embedding)
        self.embed_ln = nn.LayerNorm(h_dim)
        self.embed_timestep = nn.Embedding(max_timestep, h_dim)
        self.embed_rtg = layer_init(torch.nn.Linear(1, h_dim), std=0.01)
        self.embed_state = layer_init(torch.nn.Linear(state_dim, h_dim), std=0.01)
        
        # # discrete actions - maybe i'll not use it for risk... will look into this later
        self.embed_action_1 = torch.nn.Embedding(max_timestep, h_dim) # not act_dim
        self.embed_action_2 = layer_init(torch.nn.Linear(1, h_dim), std=0.01)
        self.embed_action = torch.nn.Linear(2*h_dim,h_dim)
        use_action_tanh = False # False for discrete actions

        # continuous actions
        #self.embed_action = torch.nn.Linear(act_dim, h_dim)
        #use_action_tanh = True # True for continuous actions
        
        ### prediction heads
        #self.predict_rtg = torch.nn.Linear(h_dim, 1)
        self.predict_rtg = layer_init(torch.nn.Linear(h_dim, act_dim), std=0.01)
        self.predict_state = layer_init(torch.nn.Linear(h_dim, state_dim), std=0.01)
        self.predict_action = nn.Sequential(
            *([layer_init(nn.Linear(h_dim, act_dim), std=0.01)] + ([nn.Tanh()] if use_action_tanh else []))
        )

        self.predict_actor_1 = nn.Sequential(
                                        *([layer_init(nn.Linear(h_dim, act_dim), std=0.01)] +
                                          ([nn.Tanh()] if use_action_tanh else []))
                                                        )
        self.predict_actor_2 = nn.Sequential(layer_init(nn.Linear(h_dim, 2), std=0.01), 
                                     nn.Softmax(dim=1),)




    


    def forward(self, timesteps, states, actions, returns_to_go):

        B, T, _ = states.shape

        time_embeddings = self.embed_timestep(timesteps)

        # time embeddings are treated similar to positional embeddings
        state_embeddings = self.embed_state(states) + time_embeddings
        #print(actions.shape)
        #print(self.embed_action(actions).squeeze().shape)
        #print(time_embeddings.shape)
        
        action_embeddings = self.embed_action(
                                            torch.concat(
                                                (self.embed_action_1(actions[:,0].squeeze()),
                                                 self.embed_action_2(actions[:,1].squeeze())),axis =1)
                                )+ time_embeddings
        #embed_action_1
        #embed_action_2
        #embed_action

        
        #action_embeddings = self.embed_action(actions.squeeze())+ time_embeddings
        returns_embeddings = self.embed_rtg(returns_to_go.float()) + time_embeddings

        # stack rtg, states and actions and reshape sequence as
        # (r1, s1, a1, r2, s2, a2 ...)
        h = torch.stack(
            (returns_embeddings, state_embeddings, action_embeddings), dim=1
        ).permute(0, 2, 1, 3).reshape(B, 3 * T, self.h_dim)

        h = self.embed_ln(h)
        
        # transformer and prediction
        h = self.transformer(h)

        # get h reshaped such that its size = (B x 3 x T x h_dim) and
        # h[:, 0, t] is conditioned on r_0, s_0, a_0 ... r_t
        # h[:, 1, t] is conditioned on r_0, s_0, a_0 ... r_t, s_t
        # h[:, 2, t] is conditioned on r_0, s_0, a_0 ... r_t, s_t, a_t
        h = h.reshape(B, T, 3, self.h_dim).permute(0, 2, 1, 3)

        # get predictions
        return_preds = self.predict_rtg(h[:,2])     # predict next rtg given r, s, a
        state_preds = self.predict_state(h[:,2])    # predict next state given r, s, a
        #action_preds = self.predict_action(h[:,1])  # predict action given r, s


        action_preds_1 = self.predict_actor_1(h[:,1])
        action_preds_2 = self.predict_actor_2(h[:,1])
        action_preds = torch.concat((action_preds_1,action_preds_2),axis =1)
    
        return state_preds, action_preds, return_preds


# custom sampler for the hero agent 
(but will not be used)

In [ ]:
from torch.utils.data.sampler import  Sampler

class YourSampler(Sampler[list[int]]):
    def __init__(self, mask,data_len):
        self.mask = mask[:,None]
        self.indices = np.arange(data_len)

    def __iter__(self):

        return (self.indices[i] for i in self.mask)

    def __len__(self):
        return len(self.mask)


sampler1 = YourSampler(torch.tensor([2,3]),4)

trainloader_sampler1 = torch.utils.data.DataLoader([([1],[2]),([5],[2]),([6],[3]),([7],[4])], batch_size=4,
                                          sampler = sampler1, shuffle=False)
[batch for batch in trainloader_sampler1]

# old dataset and definition

In [ ]:

class D4RLTrajectoryDataset(Dataset):
    def __init__(self, trajectories, context_len, rtg_scale,gamma=0.99,min_len = 10**6):

        self.context_len = context_len
        self.trajectories = trajectories

        #print(len(self.trajectories))
        # load dataset
        #with open(dataset_path, 'rb') as f:
        #    self.trajectories = pickle.load(f)

        
        # calculate min len of traj, state mean and variance
        # and returns_to_go for all traj

        
        
        states = []
        for traj in self.trajectories:
            traj_len = traj['observations'].shape[0]
            min_len = min(min_len, traj_len)
            states.append(traj['observations'])
            # calculate returns to go and rescale them
            
            traj['returns_to_go'] = discount_cumsum(traj['rewards'], gamma) / rtg_scale
            
        print(min_len)
        
        # used for input normalization
        states = torch.concatenate(states, axis=0).to(dtype = torch.float32)
        self.state_mean, self.state_std = torch.mean(states, axis=0), torch.std(states, axis=0) + 1e-6

        # normalize states
        #for traj in self.trajectories:
        #    traj['observations'] = (traj['observations'].to(dtype=torch.float32) - self.state_mean) / self.state_std


    def get_state_stats(self):
        return self.state_mean, self.state_std

    def __len__(self):
        #print(len(self.trajectories))
        return len(self.trajectories)

    def __getitem__(self, idx):
        traj = self.trajectories[idx]
        traj_len = traj['observations'].shape[0]

        if traj_len >= self.context_len:
            # sample random index to slice trajectory
            si = random.randint(0, traj_len - self.context_len)

            states = (traj['observations'][si : si + self.context_len])
            actions = (traj['actions'][si : si + self.context_len])
            reward =  (traj['rewards'][si : si + self.context_len])
            returns_to_go = (traj['returns_to_go'][si : si + self.context_len])
            
            action_masks = (traj['action_masks'][si : si + self.context_len])
            current_agent_simple = (traj['current_agent_simple'][si : si + self.context_len])
            current_agent = (traj['current_agent'][si : si + self.context_len])
            current_phase = (traj['current_phase'][si : si + self.context_len])
            current_troops_count = (traj['current_troops_count'][si : si + self.context_len])

            
            timesteps = torch.arange(start=si, end=si+self.context_len, step=1)

            # all ones since no padding
            traj_mask = torch.ones(self.context_len, dtype=torch.long)

        else:
            padding_len = self.context_len - traj_len

            # padding with zeros
            states = (traj['observations'])
            states = torch.cat([states,
                                torch.zeros(([padding_len] + list(states.shape[1:])),
                                dtype=states.dtype)], 
                               dim=0)
            
            actions = (traj['actions'])
            actions = torch.cat([actions,
                                torch.zeros(([padding_len] + list(actions.shape[1:])),
                                dtype=actions.dtype)], 
                               dim=0)
            reward = (traj['rewards'])
            reward = torch.cat([reward,
                                torch.zeros(([padding_len] + list(reward.shape[1:])),
                                dtype=reward.dtype
                                           )], 
                               dim=0)
            returns_to_go = (traj['returns_to_go'])
            returns_to_go = torch.cat([returns_to_go,
                                torch.zeros(([padding_len] + list(returns_to_go.shape[1:])),
                                dtype=returns_to_go.dtype
                                           )], 
                               dim=0)

            action_masks = (traj['action_masks'][si : si + self.context_len])
            current_agent_simple = (traj['current_agent_simple'][si : si + self.context_len])
            current_agent = (traj['current_agent'][si : si + self.context_len])
            current_phase = (traj['current_phase'][si : si + self.context_len])
            current_troops_count = (traj['current_troops_count'][si : si + self.context_len])

            action_masks = torch.cat([action_masks,
                                torch.zeros(([padding_len] + list(action_masks.shape[1:])),
                                dtype=action_masks.dtype
                                           )], 
                               dim=0)
            current_agent_simple = torch.cat([current_agent_simple,
                                torch.zeros(([padding_len] + list(current_agent_simple.shape[1:])),
                                dtype=current_agent_simple.dtype
                                           )], 
                               dim=0)
            current_agent = torch.cat([current_agent,
                                torch.zeros(([padding_len] + list(current_agent.shape[1:])),
                                dtype=current_agent.dtype
                                           )], 
                               dim=0)

            current_phase = torch.cat([current_phase,
                                torch.zeros(([padding_len] + list(current_phase.shape[1:])),
                                dtype=current_phase.dtype
                                           )], 
                               dim=0)

            current_troops_count = torch.cat([current_troops_count,
                                torch.zeros(([padding_len] + list(current_troops_count.shape[1:])),
                                dtype=current_troops_count.dtype
                                           )], 
                               dim=0)



            
            timesteps = torch.arange(start=0, end=self.context_len, step=1)

            traj_mask = torch.cat([torch.ones(traj_len, dtype=torch.long), 
                                   torch.zeros(padding_len, dtype=torch.long)], 
                                  dim=0)
            
        return  timesteps, states, actions, returns_to_go,reward, traj_mask ,action_masks,current_agent_simple,current_agent,current_phase,current_troops_count


